# Obtenção de rating do contribuinte

Com o objetivo de direcionar o modelo que ditá quais dívidas são melhores de recuperar, será realizado um rating que envolva apenas as variáveis do contribuinte.

In [1]:
import os
import dotenv
import zipfile
import pandas as pd
import numpy as np
from datetime import date

import boto3
from io import BytesIO
import pickle

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import OneHotEncoder

import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go

import pickle
import shutil

import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

import warnings
warnings.filterwarnings("ignore")

rootPath = os.getcwd()
dataPath = os.path.join(rootPath, 'data')
modelsPath = os.path.join(rootPath, 'models')
env = os.path.join(rootPath, '.env')
dotenv.load_dotenv(dotenv_path=env)

True

In [2]:
def formatar_moeda(valor):
    return locale.currency(valor, grouping=True)

def paste_intervalo(row):
    return "[" + str(row['inf']) + ", " + str(row['sup']) + ")"

def up_s3_files(dataframe, bucket_name, folder_name, file_name):
    csv_buffer = BytesIO()
    dataframe.to_csv(csv_buffer, sep=';', index=False)
    file_key_aws = folder_name + file_name
    s3_resource.Object(bucket_name, file_key_aws).put(Body=csv_buffer.getvalue())

In [3]:
dataPath

'c:\\Users\\BHN\\datascience\\divida_ativa\\data'

# 01) Importando dados

In [4]:
zip_file = os.path.join(dataPath, 'rating_igr.zip')
z = zipfile.ZipFile(zip_file)

In [5]:
def ler_bases_exportadas(nome_arquivo):
    z.extract(nome_arquivo)
    df = pd.read_csv(nome_arquivo, sep=',')
    os.remove(nome_arquivo)
    return df

In [6]:
base_notas_fiscais = ler_bases_exportadas('emissao_notas.csv')
base_conjunta = ler_bases_exportadas('conjunta_da_aberto.csv')

In [7]:
base_conjunta[['id_pessoa', 'tipo_divida']].nunique()

id_pessoa      424175
tipo_divida         2
dtype: int64

# Há CDAs que aparecem mais de 01 vez?

In [8]:
numcda = base_conjunta.groupby('cda')['tipo_divida'].count().reset_index()
numcda = numcda.sort_values(by = "tipo_divida", ascending= False)
numcda

,cda,tipo_divida
958255,6598be02c361e56a9989bae6f4af91ee1,2
2228167,ec1d4af9869491f4d8a3e5cebc255ca61,2
311699,2130cd5b5c25111f6b3a8e268c80aedf1,2
1343620,8e6c16216f5458049ceed47545e621dd1,2
1874860,c6bb566216758e359fcacff532cf50c42,2
...,...,...
832547,584c02918df1232f9ceed47545e621ddE,1
832546,584c02918df1232f9ceed47545e621dd3,1
832545,584c02918df1232f9ceed47545e621dd2,1
832544,584c02918df1232f9ceed47545e621dd1,1


In [9]:
base_conjunta.columns

Index(['cda', 'tipo_divida', 'id_pessoa', 'atividade_principal', 'situacao',
       'tipo_tributo', 'valor_pago', 'valor_tot', 'vlr_tributo', 'vlr_taxa',
       'competencia_divida', 'inscricao_divida', 'arrecadacao_divida',
       'ajuizamento_divida', 'edificacao', 'cpf_cnpj_existe', 'protesto',
       'ajuizamento', 'refis', 'deb_totais', 'deb_pagos', 'idade_divida',
       'quantidade_reparcelamento', 'da_aberto'],
      dtype='object')

# Renomeando coluna de idade da dívida

In [10]:
base_conjunta = base_conjunta.rename(columns = {'idade_divida':'anos_idade_da'})
base_conjunta

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,valor_pago,valor_tot,vlr_tributo,vlr_taxa,...,edificacao,cpf_cnpj_existe,protesto,ajuizamento,refis,deb_totais,deb_pagos,anos_idade_da,quantidade_reparcelamento,da_aberto
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,APARTAMENTO,ATIVO,IPTU,0.0,2048.73,762.08,1286.65,...,1,1,0,0,0,40.0,0.0,1.0,0,1
1,00000123c3d4731c6b3a8e268c80aedf2,mercantil,fc4b99b807fbed41,ATIVIDADES DE TELEATENDIMENTO,SUSPENSO,ISS,0.0,2515.85,0.00,2515.85,...,0,1,0,0,0,3.0,0.0,1.0,0,1
2,00000b44c5ba1e669ceed47545e621dd2,mercantil,96e8e553de69d7a4,COMERCIO VAREJISTA DE ARTIGOS DE ARMARINHO,INAPTO,ISS,0.0,1278.97,0.00,1278.97,...,0,1,0,0,0,4.0,0.0,21.0,0,0
3,000014e359592e62d8a3e5cebc255ca6E,mercantil,6dbe14da38a31dc1,Comercio varejista especializado de equipament...,ATIVO,ISS,0.0,847.02,847.02,0.00,...,0,1,0,0,0,1.0,0.0,0.0,0,1
4,0000331f601a73e52b46f673bf0c61251,imovel,870c08c252b25ad1,APARTAMENTO,ATIVO,IPTU,0.0,344.12,147.20,196.92,...,1,1,0,0,0,6.0,0.0,11.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575107,ffffd8af5760cada9fcacff532cf50c41,imovel,d411f54ad97d1f39,NaN,ATIVO,IPTU,0.0,1728.13,790.85,937.28,...,0,0,0,0,0,19.0,0.0,4.0,0,0
2575108,ffffdb721b90282718565e26094f06ee2,mercantil,51d6043a4efa1ed1,PUBLICITARIO (NÍVEL MÉDIO),ATIVO,ISS,0.0,240.77,240.77,0.00,...,0,1,0,0,0,1.0,0.0,6.0,0,0
2575109,ffffdc78aa7a90e26b3a8e268c80aedf1,imovel,9b512f210b460120,APARTAMENTO,ATIVO,IPTU,0.0,9390.43,5113.61,4276.82,...,1,1,0,1,0,30.0,0.0,25.0,0,1
2575110,fffff192f894983c9ceed47545e621dd1,imovel,dd5538d2d38803a7,CASA,ATIVO,IPTU,0.0,16594.80,11374.98,5219.82,...,1,1,0,1,0,32.0,0.0,15.0,0,1


In [11]:
formatar_moeda(base_conjunta['valor_tot'].sum())

'R$ 25.606.259.385,69'

In [12]:
base_conjunta['inscricao_divida'].max()

'2041-10-25 00:00:00'

In [13]:
base_conjunta.columns

Index(['cda', 'tipo_divida', 'id_pessoa', 'atividade_principal', 'situacao',
       'tipo_tributo', 'valor_pago', 'valor_tot', 'vlr_tributo', 'vlr_taxa',
       'competencia_divida', 'inscricao_divida', 'arrecadacao_divida',
       'ajuizamento_divida', 'edificacao', 'cpf_cnpj_existe', 'protesto',
       'ajuizamento', 'refis', 'deb_totais', 'deb_pagos', 'anos_idade_da',
       'quantidade_reparcelamento', 'da_aberto'],
      dtype='object')

# 02) Manipulando Dados

### 02.01) Valor total, pago e saldo

In [60]:
valor_tot = base_conjunta.groupby(['cda', 'tipo_divida', 'id_pessoa'])['valor_tot'].sum().to_frame().reset_index()
valor_pago = base_conjunta.groupby(['cda', 'tipo_divida', 'id_pessoa'])['valor_pago'].sum().to_frame().reset_index()
valor_aberto = base_conjunta[base_conjunta['da_aberto'] == 1].groupby(['cda', 'tipo_divida', 'id_pessoa'])['valor_tot'].sum().to_frame().reset_index()
valor_aberto = valor_aberto.rename(columns = {'valor_tot':'valor_aberto'})

valor_tot_pago = pd.merge(valor_tot, valor_pago, on = ['cda', 'tipo_divida', 'id_pessoa'], how = "left")
valor_tot_pago_aberto = pd.merge(valor_tot_pago, valor_aberto, on = ['cda', 'tipo_divida', 'id_pessoa'], how = "left")

# O que a gente esperava receber: dif_tot_pago
valor_tot_pago_aberto['dif_tot_pago'] = valor_tot_pago_aberto['valor_tot'] - valor_tot_pago_aberto['valor_pago']
# O quanto perdeu entre o que a gente esperava receber e o que foi efetivamente pago
valor_tot_pago_aberto['dif_tot_pago_aberto'] = valor_tot_pago_aberto['dif_tot_pago'] - valor_tot_pago_aberto['valor_aberto']

valor_tot_pago_aberto.sort_values(by = 'dif_tot_pago_aberto', ascending = True)

,cda,tipo_divida,id_pessoa,valor_tot,valor_pago,valor_aberto,dif_tot_pago,dif_tot_pago_aberto
2388769,fd285a26491e4db34bd01ecb8277da0b1,imovel,a815998acb8c6215,7590.99,6791.23,7485.41,799.76,-6685.65
1406307,95112448c7b350ead8a3e5cebc255ca61,imovel,a05b8e5ab1487772,2968.11,2708.55,2269.02,259.56,-2009.46
1956240,cf561e80e3ef18e79fcacff532cf50c41,imovel,a05b8e5ab1487772,3059.53,2866.91,1654.32,192.62,-1461.70
118673,0ca6a542e26c8a4518565e26094f06ee1,imovel,9be83db9e45b9e75,5010.48,4935.41,1294.99,75.07,-1219.92
486187,33b0b07752a115bcb5eae1a923f0f8701,imovel,18d6410f191932ea,2472.57,2357.60,321.72,114.97,-206.75
...,...,...,...,...,...,...,...,...
2415587,ffffd120a8f3f2799fcacff532cf50c41,imovel,934532c6c448a899,802.85,0.00,NaN,802.85,NaN
2415588,ffffd79d8aa612ac9fcacff532cf50c41,imovel,c6b16a1235d987d3,26104.42,26104.42,NaN,0.00,NaN
2415590,ffffd8af5760cada9fcacff532cf50c41,imovel,d411f54ad97d1f39,1728.13,0.00,NaN,1728.13,NaN
2415591,ffffdb721b90282718565e26094f06ee2,mercantil,51d6043a4efa1ed1,240.77,0.00,NaN,240.77,NaN


In [62]:
valor_tot_pago_aberto[~valor_tot_pago_aberto['dif_tot_pago_aberto'].isna()].sort_values(by = 'dif_tot_pago_aberto')

,cda,tipo_divida,id_pessoa,valor_tot,valor_pago,valor_aberto,dif_tot_pago,dif_tot_pago_aberto
2388769,fd285a26491e4db34bd01ecb8277da0b1,imovel,a815998acb8c6215,7590.99,6791.23,7485.41,799.76,-6685.65
1406307,95112448c7b350ead8a3e5cebc255ca61,imovel,a05b8e5ab1487772,2968.11,2708.55,2269.02,259.56,-2009.46
1956240,cf561e80e3ef18e79fcacff532cf50c41,imovel,a05b8e5ab1487772,3059.53,2866.91,1654.32,192.62,-1461.70
118673,0ca6a542e26c8a4518565e26094f06ee1,imovel,9be83db9e45b9e75,5010.48,4935.41,1294.99,75.07,-1219.92
486187,33b0b07752a115bcb5eae1a923f0f8701,imovel,18d6410f191932ea,2472.57,2357.60,321.72,114.97,-206.75
...,...,...,...,...,...,...,...,...
1216849,8104f268246d1f919ceed47545e621ddE,mercantil,b66a0dbc22eebc5a,15324729.38,1519768.82,815458.32,13804960.56,12989502.24
300803,200b26abed9c0e496b3a8e268c80aedfE,mercantil,0c38fef7c41e01ce,25663880.31,182283.98,11832898.77,25481596.33,13648697.56
2247423,ee27ee78f6c63e6318565e26094f06eeE,mercantil,60bcc46b84243022,21113432.77,1750182.07,2886797.53,19363250.70,16476453.17
273808,1d271205bb109cc89ceed47545e621ddE,mercantil,a5a62b17a2ca8a04,28440312.96,734070.01,7323732.13,27706242.95,20382510.82


In [63]:
# Caso comentado no teams
valor_tot_pago_aberto[valor_tot_pago_aberto['cda'] == '8d1f5d34b36fe9dce52281a70553db462']

,cda,tipo_divida,id_pessoa,valor_tot,valor_pago,valor_aberto,dif_tot_pago,dif_tot_pago_aberto
1331465,8d1f5d34b36fe9dce52281a70553db462,mercantil,08d5af2c269cf4ba,17339.85,760.65,6139.11,16579.2,10440.09


In [16]:
num_linhas_por_cda = base_conjunta.groupby(['cda', 'tipo_divida', 'id_pessoa'])['da_aberto'].nunique().to_frame().sort_values(by = 'da_aberto').reset_index()
num_linhas_por_cda =  num_linhas_por_cda.rename(columns = {'da_aberto':'num_da_aberto'})
num_linhas_por_cda['ja_reneg_essa_cda'] = 0
# As entradas que tem mais de um valor para da_aberto é pq foram renegociadas
num_linhas_por_cda.loc[(num_linhas_por_cda['num_da_aberto'] > 1 ), 'ja_reneg_essa_cda'] = 1 

valor_tot_pago_aberto = pd.merge(valor_tot_pago_aberto, num_linhas_por_cda, on = ['cda', 'tipo_divida', 'id_pessoa'], how = 'left')
#valor_deb_tot_pago_aberto = pd.merge(valor_tot_pago_aberto, deb_tot_pago_aberto, on = ['cda', 'tipo_divida', 'id_pessoa'], how = 'left')


In [17]:
valor_tot_pago_aberto.sort_values(by  = 'num_da_aberto')

,cda,tipo_divida,id_pessoa,valor_tot,valor_pago,valor_aberto,num_da_aberto,ja_reneg_essa_cda
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,2048.73,0.00,2048.73,1,0
1582602,a7c10d053e3a3cd42b46f673bf0c61251,imovel,bcdc26bf91553993,1798.15,1215.91,NaN,1,0
1582601,a7c10cd944b7711318565e26094f06eeD,mercantil,236f6218df052a1a,1635604.95,1470.61,NaN,1,0
1582600,a7c0f2e1ac7895f36b3a8e268c80aedf2,mercantil,a0acb0e11596c403,4361.27,0.00,NaN,1,0
1582598,a7c0ec7538487a8de52281a70553db461,imovel,09c00989a496c060,2287.52,0.00,NaN,1,0
...,...,...,...,...,...,...,...,...
187968,140ff98a415a14679989bae6f4af91ee1,imovel,a97d9bf3a03fb68a,9788.33,403.74,9384.59,2,1
747749,4f58d4fe4385c188d8a3e5cebc255ca62,mercantil,6587dcd76ef2c284,4075.01,184.70,1259.73,2,1
1614610,ab217a1f9bce0a8e9ceed47545e621ddE,mercantil,f819d9b7ffb05a56,1066956.13,19094.81,813464.70,2,1
479890,3304cee7dbda1de32b46f673bf0c61251,imovel,3a762531c5e684dd,32824.00,68.67,15609.97,2,1


In [39]:
df_conjunta = pd.merge(base_conjunta, num_linhas_por_cda, on = ['cda', 'tipo_divida', 'id_pessoa'], how = 'left')

# CDAs SEM mudanca de status
cdas_01_apar = df_conjunta[df_conjunta['num_da_aberto'] == 1]
cdas_01_apar = cdas_01_apar.sort_values(by = ['cda', 'inscricao_divida'])

# CDAs com mudanca de status
cdas_gt_01_apar = df_conjunta[df_conjunta['num_da_aberto'] > 1]
cdas_gt_01_apar = cdas_gt_01_apar.sort_values(by = ['cda', 'inscricao_divida']) # 319034 

# # CDAs com mudanca de status - ABERTO
cdas_gt_01_apar_ABERTO = cdas_gt_01_apar[cdas_gt_01_apar['da_aberto'] == 1]

reconstroi = pd.concat([cdas_01_apar, cdas_gt_01_apar_ABERTO])

reconstroi = reconstroi[['cda', 'tipo_divida', 'id_pessoa', 'atividade_principal', 'situacao',
       'tipo_tributo', 'vlr_tributo', 'vlr_taxa',
       'competencia_divida', 'inscricao_divida', 'arrecadacao_divida',
       'ajuizamento_divida', 'edificacao', 'cpf_cnpj_existe', 'protesto',
       'ajuizamento', 'refis', 'anos_idade_da',
       'quantidade_reparcelamento', 'da_aberto']]

reconstroi = pd.merge(reconstroi, valor_tot_pago_aberto, on = ['cda', 'tipo_divida', 'id_pessoa'], how = 'left')

reconstroi['perc_pago'] = np.round(reconstroi['valor_pago']/reconstroi['valor_tot'], 5)

reconstroi

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,vlr_tributo,vlr_taxa,competencia_divida,inscricao_divida,...,refis,anos_idade_da,quantidade_reparcelamento,da_aberto,valor_tot,valor_pago,valor_aberto,num_da_aberto,ja_reneg_essa_cda,perc_pago
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,APARTAMENTO,ATIVO,IPTU,762.08,1286.65,2021-01-01,2022-09-07,...,0,1.0,0,1,2048.73,0.00,2048.73,1,0,0.00000
1,00000123c3d4731c6b3a8e268c80aedf2,mercantil,fc4b99b807fbed41,ATIVIDADES DE TELEATENDIMENTO,SUSPENSO,ISS,0.00,2515.85,2019-01-01,2022-12-30,...,0,1.0,0,1,2515.85,0.00,2515.85,1,0,0.00000
2,00000b44c5ba1e669ceed47545e621dd2,mercantil,96e8e553de69d7a4,COMERCIO VAREJISTA DE ARTIGOS DE ARMARINHO,INAPTO,ISS,0.00,1278.97,2001-08-03,2002-09-07,...,0,21.0,0,0,1278.97,0.00,NaN,1,0,0.00000
3,000014e359592e62d8a3e5cebc255ca6E,mercantil,6dbe14da38a31dc1,Comercio varejista especializado de equipament...,ATIVO,ISS,847.02,0.00,2022-12-01,2023-06-15,...,0,0.0,0,1,847.02,0.00,847.02,1,0,0.00000
4,0000331f601a73e52b46f673bf0c61251,imovel,870c08c252b25ad1,APARTAMENTO,ATIVO,IPTU,147.20,196.92,2010-01-01,2012-10-17,...,0,11.0,0,0,344.12,0.00,NaN,1,0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2415590,fffe69ada60a04979ceed47545e621dd1,imovel,910e31fa556db12c,CASA,ATIVO,IPTU,15601.48,0.00,2015-07-16,2015-08-10,...,0,8.0,0,1,33597.69,919.49,15601.48,2,1,0.02737
2415591,fffe75c5e436f4e69fcacff532cf50c42,mercantil,eefe16fde5758b38,FOTOCÓPIAS,INAPTO,ISS,2077.97,0.00,2018-12-11,2018-12-20,...,0,5.0,2,1,6173.14,578.55,2077.97,2,1,0.09372
2415592,fffe8ed282868b7d18565e26094f06ee1,imovel,1eb4adad813da472,CASA,ATIVO,IPTU,7150.17,0.00,2015-10-08,2015-11-10,...,0,8.0,2,1,16098.88,696.05,7150.17,2,1,0.04324
2415593,fffeb819f46062317c8fa7a67092eaab2,mercantil,52da39231b917f91,CARGA E DESCARGA,INAPTO,ISS,2864.02,0.00,2016-12-01,2016-12-09,...,0,7.0,0,1,3229.55,365.53,2864.02,2,1,0.11318


In [40]:
reconstroi[reconstroi['cda'] == '0000a43dae672b166b3a8e268c80aedf1']

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,vlr_tributo,vlr_taxa,competencia_divida,inscricao_divida,...,refis,anos_idade_da,quantidade_reparcelamento,da_aberto,valor_tot,valor_pago,valor_aberto,num_da_aberto,ja_reneg_essa_cda,perc_pago
2256078,0000a43dae672b166b3a8e268c80aedf1,imovel,f5fe60dbb517e68a,APARTAMENTO,ATIVO,IPTU,2094.28,0.0,2022-08-24,2022-08-29,...,0,1.0,0,1,3786.64,1692.36,2094.28,2,1,0.44693


In [52]:
# CDAs que vamos treinar
da_aberto_0 = reconstroi[reconstroi['da_aberto'] == 0]
da_aberto_0

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,vlr_tributo,vlr_taxa,competencia_divida,inscricao_divida,...,refis,anos_idade_da,quantidade_reparcelamento,da_aberto,valor_tot,valor_pago,valor_aberto,num_da_aberto,ja_reneg_essa_cda,perc_pago
2,00000b44c5ba1e669ceed47545e621dd2,mercantil,96e8e553de69d7a4,COMERCIO VAREJISTA DE ARTIGOS DE ARMARINHO,INAPTO,ISS,0.00,1278.97,2001-08-03,2002-09-07,...,0,21.0,0,0,1278.97,0.00,NaN,1,0,0.0000
4,0000331f601a73e52b46f673bf0c61251,imovel,870c08c252b25ad1,APARTAMENTO,ATIVO,IPTU,147.20,196.92,2010-01-01,2012-10-17,...,0,11.0,0,0,344.12,0.00,NaN,1,0,0.0000
7,000040eda866e3d19ceed47545e621dd1,imovel,9f3bac8718dac1fa,LOJA,ATIVO,IPTU,10779.87,0.00,2015-01-13,2015-01-30,...,0,15.0,2,0,10779.87,5438.40,NaN,1,0,0.5045
8,000040eda866e3d19ceed47545e621dd2,mercantil,2f8ae1b9606267b4,INSTRUTOR DE TREINAMENTOS,INAPTO,ISS,328.63,0.00,2004-07-01,2008-10-05,...,0,15.0,0,0,328.63,0.00,NaN,1,0,0.0000
10,0000521b64bf28c8e52281a70553db461,imovel,b5df816e1786f0a3,CASA,ATIVO,IPTU,245.50,245.33,2014-01-01,2016-05-14,...,0,7.0,0,0,490.83,0.00,NaN,1,0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256071,ffffd120a8f3f2799fcacff532cf50c41,imovel,934532c6c448a899,CASA,ATIVO,IPTU,617.95,184.90,2013-01-01,2014-07-26,...,0,9.0,0,0,802.85,0.00,NaN,1,0,0.0000
2256072,ffffd79d8aa612ac9fcacff532cf50c41,imovel,c6b16a1235d987d3,APARTAMENTO,ATIVO,IPTU,22190.68,3913.74,2021-01-01,2022-04-07,...,0,1.0,0,0,26104.42,26104.42,NaN,1,0,1.0000
2256073,ffffd8af5760cada9fcacff532cf50c41,imovel,d411f54ad97d1f39,NaN,ATIVO,IPTU,790.85,937.28,2017-01-01,2019-06-28,...,0,4.0,0,0,1728.13,0.00,NaN,1,0,0.0000
2256074,ffffdb721b90282718565e26094f06ee2,mercantil,51d6043a4efa1ed1,PUBLICITARIO (NÍVEL MÉDIO),ATIVO,ISS,240.77,0.00,2015-01-01,2017-10-22,...,0,6.0,0,0,240.77,0.00,NaN,1,0,0.0000


In [53]:
da_aberto_0[da_aberto_0['perc_pago'] == 0] # 989873  pagaram 0

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,vlr_tributo,vlr_taxa,competencia_divida,inscricao_divida,...,refis,anos_idade_da,quantidade_reparcelamento,da_aberto,valor_tot,valor_pago,valor_aberto,num_da_aberto,ja_reneg_essa_cda,perc_pago
2,00000b44c5ba1e669ceed47545e621dd2,mercantil,96e8e553de69d7a4,COMERCIO VAREJISTA DE ARTIGOS DE ARMARINHO,INAPTO,ISS,0.00,1278.97,2001-08-03,2002-09-07,...,0,21.0,0,0,1278.97,0.0,NaN,1,0,0.0
4,0000331f601a73e52b46f673bf0c61251,imovel,870c08c252b25ad1,APARTAMENTO,ATIVO,IPTU,147.20,196.92,2010-01-01,2012-10-17,...,0,11.0,0,0,344.12,0.0,NaN,1,0,0.0
8,000040eda866e3d19ceed47545e621dd2,mercantil,2f8ae1b9606267b4,INSTRUTOR DE TREINAMENTOS,INAPTO,ISS,328.63,0.00,2004-07-01,2008-10-05,...,0,15.0,0,0,328.63,0.0,NaN,1,0,0.0
10,0000521b64bf28c8e52281a70553db461,imovel,b5df816e1786f0a3,CASA,ATIVO,IPTU,245.50,245.33,2014-01-01,2016-05-14,...,0,7.0,0,0,490.83,0.0,NaN,1,0,0.0
11,000057f68f7b77276b3a8e268c80aedf1,imovel,e3df827c83c5efa8,CASA,ATIVO,IPTU,555.67,592.39,2016-01-01,2018-07-07,...,0,5.0,0,0,1148.06,0.0,NaN,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256067,ffffafbbcfae89607c8fa7a67092eaab2,mercantil,172160e5eb98bf3c,COMÉRCIO VAREJISTA DE BRINQUEDOS E ARTIGOS REC...,INAPTO,ISS,0.00,2118.05,2014-07-01,2016-05-14,...,0,7.0,0,0,2118.05,0.0,NaN,1,0,0.0
2256068,ffffb75e1625adb04bd01ecb8277da0b2,mercantil,48dde0f302dd0b90,VENDEDOR COMISSIONADO,BAIXADO,ISS,241.62,0.00,2011-01-01,2013-10-19,...,0,10.0,0,0,241.62,0.0,NaN,1,0,0.0
2256071,ffffd120a8f3f2799fcacff532cf50c41,imovel,934532c6c448a899,CASA,ATIVO,IPTU,617.95,184.90,2013-01-01,2014-07-26,...,0,9.0,0,0,802.85,0.0,NaN,1,0,0.0
2256073,ffffd8af5760cada9fcacff532cf50c41,imovel,d411f54ad97d1f39,NaN,ATIVO,IPTU,790.85,937.28,2017-01-01,2019-06-28,...,0,4.0,0,0,1728.13,0.0,NaN,1,0,0.0


In [54]:
da_aberto_0[da_aberto_0['perc_pago'] == 1] # 240883  pagaram tudo

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,vlr_tributo,vlr_taxa,competencia_divida,inscricao_divida,...,refis,anos_idade_da,quantidade_reparcelamento,da_aberto,valor_tot,valor_pago,valor_aberto,num_da_aberto,ja_reneg_essa_cda,perc_pago
21,0000905d980177f36b3a8e268c80aedf1,imovel,f08282eba436ba83,APARTAMENTO,ATIVO,IPTU,272.20,256.29,2018-01-01,2020-12-31,...,0,3.0,0,0,528.49,528.49,NaN,1,0,1.0
22,0000a43dae672b166b3a8e268c80aedf2,mercantil,94ba02ded51129a5,RESTAURANTES E SIMILARES,ATIVO,ISS,1928.97,0.00,2022-08-23,2022-12-29,...,0,1.0,1,0,1928.97,1928.97,NaN,1,0,1.0
30,0000ea1f35374e602b46f673bf0c61251,imovel,c92d35ce72ee3363,APARTAMENTO,ATIVO,IPTU,241.56,181.68,2015-01-01,2017-10-21,...,0,6.0,0,0,423.24,423.24,NaN,1,0,1.0
31,0000ea1f35374e602b46f673bf0c61252,mercantil,64c2995dab96f34c,ENSINO FUNDAMENTAL,SUSPENSO,ISS,0.00,221.78,2015-07-01,2017-10-21,...,0,6.0,0,0,221.78,221.78,NaN,1,0,1.0
37,000103c147488e019ceed47545e621dd1,imovel,323a21690f5565f3,APARTAMENTO,ATIVO,IPTU,508.75,0.00,2006-05-09,2006-11-30,...,0,17.0,1,0,508.75,508.75,NaN,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256041,fffebfc302df64ae6b3a8e268c80aedf1,imovel,8455f043f0c21d9f,APARTAMENTO,ATIVO,IPTU,359.25,465.29,2016-01-01,2018-07-07,...,0,5.0,0,0,824.54,824.54,NaN,1,0,1.0
2256042,fffed0c5b3db8c959ceed47545e621dd1,imovel,e66845e1443aecc9,APARTAMENTO,ATIVO,IPTU,172.22,0.00,2009-01-05,2009-04-10,...,0,14.0,1,0,172.22,172.22,NaN,1,0,1.0
2256062,ffff9425053e623a6b3a8e268c80aedf1,imovel,73f815530427f32a,CASA,EM PROCESSO DE BAIXA,IPTU,44182.66,0.00,2013-05-23,2017-10-30,...,0,13.0,1,0,111112.73,111112.73,NaN,1,0,1.0
2256072,ffffd79d8aa612ac9fcacff532cf50c41,imovel,c6b16a1235d987d3,APARTAMENTO,ATIVO,IPTU,22190.68,3913.74,2021-01-01,2022-04-07,...,0,1.0,0,0,26104.42,26104.42,NaN,1,0,1.0


In [56]:
da_aberto_0[(da_aberto_0['perc_pago'] > 0) & (da_aberto_0['perc_pago'] < 1)] # 217996 pagaram algo

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,vlr_tributo,vlr_taxa,competencia_divida,inscricao_divida,...,refis,anos_idade_da,quantidade_reparcelamento,da_aberto,valor_tot,valor_pago,valor_aberto,num_da_aberto,ja_reneg_essa_cda,perc_pago
7,000040eda866e3d19ceed47545e621dd1,imovel,9f3bac8718dac1fa,LOJA,ATIVO,IPTU,10779.87,0.00,2015-01-13,2015-01-30,...,0,15.0,2,0,10779.87,5438.40,NaN,1,0,0.50450
16,00006f113b5e15229ceed47545e621dd1,imovel,d75eaf7b65185a9d,CASA,ATIVO,IPTU,11929.83,372.43,2011-06-21,2015-06-30,...,0,18.0,1,0,12302.26,3522.19,NaN,1,0,0.28630
34,0000f7ef8630f89c9ceed47545e621dd1,imovel,e5e6418c7b7fc452,APARTAMENTO,ATIVO,IPTU,7154.44,0.00,2006-01-12,2008-10-30,...,0,17.0,1,0,7154.44,2161.46,NaN,1,0,0.30211
50,000144e266882447b5eae1a923f0f8701,imovel,be9dc6c861eb9bd8,CASA,ATIVO,IPTU,11909.04,0.00,2017-04-20,2019-07-10,...,0,13.0,2,0,11909.04,6022.24,NaN,1,0,0.50569
51,00015636fdd1fd409ceed47545e621dd1,imovel,c03023fbd30ac455,CASA,ATIVO,IPTU,28660.00,4133.63,2015-01-26,2015-12-30,...,0,19.0,4,0,32793.63,2623.58,NaN,1,0,0.08000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256031,fffe8ed282868b7d18565e26094f06ee2,mercantil,fd4c2de516d26890,MANUTENÇÃO E REPARAÇÃO DE MÁQUINAS E EQUIP P/ ...,ATIVO,ISS,2782.36,0.00,2013-03-07,2013-08-30,...,0,15.0,2,0,2782.36,1660.40,NaN,1,0,0.59676
2256046,fffeddffd0953ec44bd01ecb8277da0b2,mercantil,53bb94d05f86140a,REPRESENTANTES COMERCIAIS E AGENTES DO COMÉRCI...,ATIVO,ISS,11100.05,0.00,2016-08-16,2016-10-21,...,0,9.0,3,0,15827.73,8636.99,NaN,1,0,0.54569
2256051,ffff0e3e56845cdf9ceed47545e621dd1,imovel,60c3cd78362df548,CASA,ATIVO,IPTU,3056.41,0.00,2009-10-02,2010-03-10,...,0,15.0,2,0,3056.41,1452.77,NaN,1,0,0.47532
2256052,ffff29b6257a2a546b3a8e268c80aedf1,imovel,e2aa2372ea8423c3,LOJA,ATIVO,IPTU,52410.81,0.00,2013-10-09,2014-12-30,...,0,10.0,1,0,52410.81,19827.77,NaN,1,0,0.37831


In [57]:
(240883 + 217996)/ (240883 + 989873 + 217996 ) # 31% dos dados de da aberto = 0 tem alguma informação de pagamento

0.3167408914707279

# 03) Criando variáveis para clusterização

In [156]:
print("Gerando variáveis para identificação dos grupos de contribuintes")

dados_pessoas = reconstroi[['tipo_divida', 'cda', 'id_pessoa', 'situacao', 'cpf_cnpj_existe', 'edificacao', 'deb_totais', 'deb_pagos', 
                                'valor_tot', 'valor_pago',      
                                'quantidade_reparcelamento', 'anos_idade_da',
                                'ja_reneg_essa_cda']]


dados_pessoas.dropna(subset = ['id_pessoa'], inplace = True)

dados_pessoas

Gerando variáveis para identificação dos grupos de contribuintes


,tipo_divida,cda,id_pessoa,situacao,cpf_cnpj_existe,edificacao,deb_totais,deb_pagos,valor_tot,valor_pago,quantidade_reparcelamento,anos_idade_da,ja_reneg_essa_cda
0,imovel,00000123c3d4731c6b3a8e268c80aedf1,ac28642d7c82b33f,ATIVO,1,1,40.0,0.0,2048.73,0.00,0,1.0,0
1,mercantil,00000123c3d4731c6b3a8e268c80aedf2,fc4b99b807fbed41,SUSPENSO,1,0,3.0,0.0,2515.85,0.00,0,1.0,0
2,mercantil,00000b44c5ba1e669ceed47545e621dd2,96e8e553de69d7a4,INAPTO,1,0,4.0,0.0,1278.97,0.00,0,21.0,0
3,mercantil,000014e359592e62d8a3e5cebc255ca6E,6dbe14da38a31dc1,ATIVO,1,0,1.0,0.0,847.02,0.00,0,0.0,0
4,imovel,0000331f601a73e52b46f673bf0c61251,870c08c252b25ad1,ATIVO,1,1,6.0,0.0,344.12,0.00,0,11.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2415590,imovel,fffe69ada60a04979ceed47545e621dd1,910e31fa556db12c,ATIVO,1,1,1.0,0.0,33597.69,919.49,0,8.0,1
2415591,mercantil,fffe75c5e436f4e69fcacff532cf50c42,eefe16fde5758b38,INAPTO,1,0,1.0,0.0,6173.14,578.55,2,5.0,1
2415592,imovel,fffe8ed282868b7d18565e26094f06ee1,1eb4adad813da472,ATIVO,1,1,1.0,0.0,16098.88,696.05,2,8.0,1
2415593,mercantil,fffeb819f46062317c8fa7a67092eaab2,52da39231b917f91,INAPTO,1,0,1.0,0.0,3229.55,365.53,0,7.0,1


In [165]:
#reconstroi['perc_pago'] = np.round(reconstroi['valor_pago']/reconstroi['valor_tot'], 5)
#xx = reconstroi[(reconstroi['perc_pago'] == 0) & (reconstroi['da_aberto'] == 0)]
#xx.sort_values(by = 'perc_pago')

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,vlr_tributo,vlr_taxa,competencia_divida,inscricao_divida,...,quantidade_reparcelamento,da_aberto,deb_totais,deb_pagos,valor_tot,valor_pago,valor_aberto,num_da_aberto,ja_reneg_essa_cda,perc_pago
2,00000b44c5ba1e669ceed47545e621dd2,mercantil,96e8e553de69d7a4,COMERCIO VAREJISTA DE ARTIGOS DE ARMARINHO,INAPTO,ISS,0.00,1278.97,2001-08-03,2002-09-07,...,0,0,4.0,0.0,1278.97,0.0,NaN,1,0,0.0
1503322,aa8ff48381a23eb96b3a8e268c80aedf2,mercantil,620c8799ea48325e,ENGENHEIRO,INAPTO,ISS,228.73,0.00,1996-08-03,1998-01-06,...,0,0,3.0,0.0,228.73,0.0,NaN,1,0,0.0
1503325,aa9003a259534bcc2b46f673bf0c61252,mercantil,47ebc8abb631e7c6,PINTOR,INAPTO,ISS,336.46,0.00,2012-07-01,2014-10-25,...,0,0,2.0,0.0,336.46,0.0,NaN,1,0,0.0
1503327,aa9019aeca37f950e52281a70553db461,imovel,09b14d8af84d5659,NaN,EM PROCESSO DE BAIXA,IPTU,1223.39,1016.57,2014-01-01,2016-05-14,...,0,0,20.0,0.0,2239.96,0.0,NaN,1,0,0.0
1503328,aa9019aeca37f950e52281a70553db462,mercantil,1521b797c62c941e,ATIVIDADES DE ORGANIZAÇÕES SINDICAIS,INAPTO,ISS,0.00,1676.60,2014-07-01,2016-05-14,...,0,0,2.0,0.0,1676.60,0.0,NaN,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752641,557a157ba4c4a7a618565e26094f06ee2,mercantil,b16102bb5c41fc5c,REPRESENT COM E AGENTES DO COM DE PEÇAS E ACES...,ATIVO,ISS,0.00,1447.28,2013-07-01,2014-10-25,...,0,0,2.0,0.0,1447.28,0.0,NaN,1,0,0.0
752643,557a21839982255218565e26094f06ee2,mercantil,39ccdee8f4e60194,CABELEIREIRO,INAPTO,ISS,1189.74,0.00,2010-07-01,2012-09-15,...,0,0,6.0,0.0,1189.74,0.0,NaN,1,0,0.0
752644,557a3e9bf6cd79629fcacff532cf50c41,imovel,1d1b3ad54f763d70,APARTAMENTO,ATIVO,IPTU,433.65,429.36,2017-01-01,2019-06-28,...,0,0,13.0,0.0,863.01,0.0,NaN,1,0,0.0
752740,557cd0c7957b3b5e9989bae6f4af91ee2,mercantil,0a75cdae652c1549,REPRESENTANTES COMERCIAIS E AGENTES DO COM DE ...,INAPTO,ISS,0.00,8758.42,1995-08-03,1996-09-06,...,0,0,6.0,0.0,8758.42,0.0,NaN,1,0,0.0


In [157]:
xx = dados_pessoas[dados_pessoas['ja_reneg_essa_cda'] == 1].groupby(['id_pessoa', 'tipo_divida'])['cda'].nunique().to_frame().reset_index()
xx

,id_pessoa,tipo_divida,cda
0,0002644d836d4141,imovel,2
1,0005d30ef81c01c4,imovel,2
2,000722117d63c81c,imovel,5
3,0007a35a19982d05,imovel,7
4,0007f1cd58c2dfb2,mercantil,1
...,...,...,...
70359,fffa5f445cc41ff7,imovel,1
70360,fffbf81ab1d18c5a,imovel,1
70361,fffc0651dd11f426,imovel,6
70362,fffc2ed4460bead4,mercantil,1


# VAMOS ANALISAR?

In [159]:
bs_conj = reconstroi[(reconstroi['id_pessoa'] == '000722117d63c81c') & (reconstroi['tipo_divida'] == 'imovel')].sort_values(by = 'cda')
bs_conj = bs_conj[['cda', 'tipo_divida', 'id_pessoa', 'inscricao_divida', 'valor_pago',	'valor_tot', 'vlr_tributo',	'vlr_taxa', 'da_aberto']]
ja_reneg_essa_cda = dados_pessoas.loc[dados_pessoas['id_pessoa']  == '000722117d63c81c', ['cda', 'tipo_divida', 'id_pessoa', 'ja_reneg_essa_cda']]


pd.merge(bs_conj, ja_reneg_essa_cda, on = ['cda', 'tipo_divida', 'id_pessoa'], how = 'left')

,cda,tipo_divida,id_pessoa,inscricao_divida,valor_pago,valor_tot,vlr_tributo,vlr_taxa,da_aberto,ja_reneg_essa_cda
0,0a8cab6bbf311e4818565e26094f06ee1,imovel,000722117d63c81c,2019-06-28,6294.66,6294.66,4991.94,1302.72,0,0
1,0c0939d83041f6d89989bae6f4af91ee1,imovel,000722117d63c81c,2023-02-25,536.02,8363.95,7827.93,0.00,1,1
2,1ba6b58b877b50507c8fa7a67092eaab1,imovel,000722117d63c81c,2023-02-25,1090.95,33690.38,15950.66,0.00,1,1
3,3cce22480c33111f2b46f673bf0c61251,imovel,000722117d63c81c,2023-01-25,1463.74,30676.91,14069.18,0.00,1,1
4,423e72500b92021d9fcacff532cf50c41,imovel,000722117d63c81c,2023-02-25,692.46,22282.12,10033.79,0.00,1,1
5,429180603ea716fa9ceed47545e621dd1,imovel,000722117d63c81c,2011-08-25,2606.10,7676.39,7676.39,0.00,0,0
6,4d512a19f62ed6eb9ceed47545e621dd1,imovel,000722117d63c81c,2013-08-28,12865.82,56151.37,56151.37,0.00,0,0
7,527a26eca1994b77e52281a70553db461,imovel,000722117d63c81c,2014-06-30,12352.54,178643.60,172128.03,6515.57,0,0
8,5a1e3841d7dccfcd6b3a8e268c80aedf1,imovel,000722117d63c81c,2016-05-14,5075.84,5075.84,4315.54,760.30,0,0
9,81bf93ef7d6b92987c8fa7a67092eaab1,imovel,000722117d63c81c,2023-01-25,1381.34,28871.71,13244.84,0.00,1,1


In [160]:
reconstroi[reconstroi['cda'] == '0c0939d83041f6d89989bae6f4af91ee1']

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,vlr_tributo,vlr_taxa,competencia_divida,inscricao_divida,...,anos_idade_da,quantidade_reparcelamento,da_aberto,deb_totais,deb_pagos,valor_tot,valor_pago,valor_aberto,num_da_aberto,ja_reneg_essa_cda
2263671,0c0939d83041f6d89989bae6f4af91ee1,imovel,000722117d63c81c,APARTAMENTO,ATIVO,IPTU,7827.93,0.0,2023-01-20,2023-02-25,...,0.0,0,1,1.0,0.0,8363.95,536.02,7827.93,2,1


In [161]:
base_conjunta[base_conjunta['cda'] == '0c0939d83041f6d89989bae6f4af91ee1']

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,valor_pago,valor_tot,vlr_tributo,vlr_taxa,...,edificacao,cpf_cnpj_existe,protesto,ajuizamento,refis,deb_totais,deb_pagos,anos_idade_da,quantidade_reparcelamento,da_aberto
120411,0c0939d83041f6d89989bae6f4af91ee1,imovel,000722117d63c81c,APARTAMENTO,ATIVO,IPTU,536.02,536.02,536.02,0.0,...,1,1,0,0,0,4.0,4.0,0.0,0,0
120412,0c0939d83041f6d89989bae6f4af91ee1,imovel,000722117d63c81c,APARTAMENTO,ATIVO,IPTU,0.00,7827.93,7827.93,0.0,...,1,1,0,0,0,1.0,0.0,0.0,0,1


In [99]:
536.02 / (536.02 + 7827.93)

0.06408694456566573

In [ ]:
(536.02 + 7827.93) = Total

In [ ]:
7827.93 = Saldo

# Que tipo de agrupamento faremos?

--------------

In [91]:
    print('AGRUP_COM_FREQ_PESSOAS == 1')

    # Calculo que apresenta quantas cdas o contribuinte tem
    frequencia_da_pessoa = reconstroi.groupby(['id_pessoa', 'tipo_divida'])['cda'].nunique().to_frame().reset_index()
    total_reparcelamentos_pessoa = reconstroi.groupby(['id_pessoa', 'tipo_divida'])['quantidade_reparcelamento'].sum().to_frame().reset_index()
    total_debitos_pessoa = reconstroi.groupby(['id_pessoa', 'tipo_divida'])['deb_totais'].sum().to_frame().reset_index()
    debitos_pagos_pessoa = reconstroi.groupby(['id_pessoa', 'tipo_divida'])['deb_pagos'].sum().to_frame().reset_index()
    valor_total_pessoa = reconstroi.groupby(['id_pessoa', 'tipo_divida'])['valor_tot'].sum().to_frame().reset_index()
    valor_pago_pessoa = reconstroi.groupby(['id_pessoa', 'tipo_divida'])['valor_pago'].sum().to_frame().reset_index()

    valor_total_pessoa_anteriormente = reconstroi.groupby(['id_pessoa', 'tipo_divida'])['valor_tot_anteriormente'].sum().to_frame().reset_index()
    valor_pago_pessoa_anteriormente = reconstroi.groupby(['id_pessoa', 'tipo_divida'])['valor_pago_anteriormente'].sum().to_frame().reset_index()

    ja_reneg_essa_cda = reconstroi[reconstroi['ja_reneg_essa_cda'] == 1].groupby(['id_pessoa', 'tipo_divida'])['cda'].nunique().to_frame().reset_index()

    # Edificacao
    edificacao = reconstroi[['tipo_divida',	'id_pessoa', 'edificacao']]
    edificacao = edificacao.drop_duplicates()

    # Situacao
    situacao = reconstroi[['tipo_divida', 'id_pessoa', 'situacao']]
    situacao = situacao.drop_duplicates()

    # cpf_cnpj_existe
    cpf_cnpj_existe = reconstroi[['tipo_divida', 'id_pessoa', 'cpf_cnpj_existe']]
    cpf_cnpj_existe = cpf_cnpj_existe.drop_duplicates()

    ## ----------------------------------- ##
    ## -- Criando um objeto por pessoas -- ##
    ## ----------------------------------- ##
    CHAVE = ['id_pessoa', 'tipo_divida']
    freq_repal = pd.merge(frequencia_da_pessoa, total_reparcelamentos_pessoa, on = CHAVE, how = 'left') # id_pessoa	tipo_divida	cda	quantidade_reparcelamento
    freq_repal = freq_repal.rename(columns = {'cda':'num_dist_cda'})

    debitos = pd.merge(total_debitos_pessoa, debitos_pagos_pessoa, on = CHAVE, how = 'outer') # id_pessoa	tipo_divida	deb_totais	deb_pagos
    valor = pd.merge(valor_total_pessoa, valor_pago_pessoa, on = CHAVE, how = 'outer') # id_pessoa	tipo_divida	valor_tot	valor_pago
    notas_edif = pd.merge(base_notas_fiscais, edificacao, on = 'id_pessoa', how = 'outer') # id_pessoa	qtd_notas_2anos	tipo_divida	edificacao
    situ_doc = pd.merge(situacao, cpf_cnpj_existe, on = CHAVE, how = 'outer') # tipo_divida	id_pessoa	situacao	cpf_cnpj_existe

    freq_repal_debitos = pd.merge(freq_repal, debitos, on = CHAVE, how = 'left')
    valor_notas_edif = pd.merge(valor, notas_edif, on = CHAVE, how = 'left')

    freq_repal_debitos_valor_notas_edif = pd.merge(freq_repal_debitos, valor_notas_edif, on = CHAVE, how = 'left')

    # valor pago e total anteriormente
    valor_anteriormente = pd.merge(valor_total_pessoa_anteriormente, valor_pago_pessoa_anteriormente, on = CHAVE, how = 'outer') # id_pessoa	tipo_divida	valor_tot	valor_pago

    valor_anteriormente_ja_reneg_essa_cda = pd.merge(valor_anteriormente, ja_reneg_essa_cda, on = CHAVE, how = 'left')

    freq_repal_debitos_valor_notas_edif_ja_reneg_essa_cda = pd.merge(freq_repal_debitos_valor_notas_edif, valor_anteriormente_ja_reneg_essa_cda, on = CHAVE, how = 'left')

    pessoas = pd.merge(freq_repal_debitos_valor_notas_edif_ja_reneg_essa_cda, situ_doc, on = CHAVE, how = 'left')

    # Substituindo por zero os valores nulos
    pessoas['qtd_notas_2anos'] = pessoas['qtd_notas_2anos'].fillna(0)
    pessoas['edificacao'] = pessoas['edificacao'].fillna(0)

    # MERCANTIL
    pessoas.loc[(pessoas['tipo_divida'] == 'mercantil' ) & (pessoas['qtd_notas_2anos'] > 0) & (pessoas['situacao'] == 'ATIVO'), 'situacao_ativa'] = 2
    pessoas.loc[(pessoas['tipo_divida'] == 'mercantil' ) & (pessoas['qtd_notas_2anos'] > 0) & (pessoas['situacao'] != 'ATIVO'), 'situacao_ativa'] = 1
    pessoas.loc[(pessoas['tipo_divida'] == 'mercantil' ) & (pessoas['qtd_notas_2anos'] == 0) & (pessoas['situacao'] == 'ATIVO'), 'situacao_ativa'] = 1
    pessoas.loc[(pessoas['tipo_divida'] == 'mercantil' ) & (pessoas['qtd_notas_2anos'] == 0) & (pessoas['situacao'] != 'ATIVO'), 'situacao_ativa'] = 0

        
    # IMOVEL
    pessoas.loc[(pessoas['tipo_divida'] == 'imovel' ) & (pessoas['edificacao'] == 1), 'situacao_ativa'] = 2
    pessoas.loc[(pessoas['tipo_divida'] == 'imovel' ) & (pessoas['edificacao'] == 0) & (pessoas['cpf_cnpj_existe'] == 1), 'situacao_ativa'] = 1
    pessoas.loc[(pessoas['tipo_divida'] == 'imovel' ) & (pessoas['edificacao'] == 0) & (pessoas['cpf_cnpj_existe'] == 0), 'situacao_ativa'] = 0


    # Imputando 0
    pessoas['status_situacao'] = pessoas['situacao_ativa'] + pessoas['cpf_cnpj_existe']
    pessoas.loc[pessoas['situacao_ativa'] == 0, 'status_situacao'] = 0
    #pessoas.loc[pessoas['cpf_cnpj_existe'] == 0, 'status_situacao'] = 0


    # Faz o calculo do historico de pagamento
    pessoas.loc[(pessoas['deb_totais'].isna()) | (pessoas['deb_totais'] == 0) , 'deb_totais'] = 1
    pessoas.loc[(pessoas['valor_tot'].isna()) | (pessoas['valor_tot'] == 0) , 'valor_tot'] = 1

    pessoas['historico_pagamento_em_qtd'] = pessoas['deb_pagos'] / (pessoas['deb_totais'])
    pessoas['historico_pagamento_em_valor'] = pessoas['valor_pago'] / (pessoas['valor_tot'])
    pessoas['historico_pagamento_em_valor_anteriormente'] = pessoas['valor_pago_anteriormente'] / (pessoas['valor_tot_anteriormente'])
    pessoas = pessoas.sort_values(by = 'historico_pagamento_em_valor', ascending = False)
    
    ## ---------------------------------------- ##
    ## -- Monta dataframe para clusterização -- ##
    ## ---------------------------------------- ##

    # Filtrando contribuintes com mais de uma CDA
    df_pipe_cluster = pessoas.query("num_dist_cda > 1")
    

    df_pipe_cluster = df_pipe_cluster[['id_pessoa',
                                       'tipo_divida',
                                       'status_situacao',
                                       'num_dist_cda',              # nova (antigo frequencia_da_pessoa)
                                       'quantidade_reparcelamento', # nova
                                       'historico_pagamento_em_qtd', 
                                       'historico_pagamento_em_valor',
                                       'historico_pagamento_em_valor_anteriormente']]
    #df_pipe_cluster = df_pipe_cluster.set_index(['id_pessoa', 'tipo_divida'])


AGRUP_COM_FREQ_PESSOAS == 1


In [18]:
#pessoas[pessoas['id_pessoa'] == '000032f6d93a0abd'] # 000032f6d93a0abd # def2e7cb1e2f6ae1
pessoas

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,valor_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,situacao_ativa,status_situacao,historico_pagamento_em_qtd,historico_pagamento_em_valor
247206,7fcd58cfe066e299,mercantil,1,0,4.0,4.0,533.37,533.37,0.0,0.0,ATIVO,1,1.0,2.0,1.0,1.0
467137,f1e9dde2a3053d51,imovel,4,0,79.0,79.0,7987.43,7987.43,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0
183548,5f225ff188d71bf6,imovel,1,0,12.0,12.0,454.28,454.28,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0
183545,5f21929a71c17a08,mercantil,1,0,1.0,1.0,550.47,550.47,0.0,0.0,BAIXADO,1,0.0,0.0,1.0,1.0
49086,196d80b1b82edbc0,mercantil,2,2,8.0,8.0,493.97,493.97,0.0,0.0,SUSPENSO,1,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212557,6dec5209fbc2f813,imovel,4,0,136.0,0.0,14185.80,0.00,0.0,1.0,ATIVO,1,2.0,3.0,0.0,0.0
212556,6dec36900dea378c,mercantil,7,0,52.0,0.0,19190.68,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0
212552,6debc091e4bf4d84,mercantil,2,0,6.0,0.0,14930.45,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0
212551,6debbb4cc485f25f,mercantil,8,0,25.0,0.0,29849.66,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0


In [92]:
df_pipe_cluster[df_pipe_cluster['id_pessoa'] == '000722117d63c81c']

,id_pessoa,tipo_divida,status_situacao,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_qtd,historico_pagamento_em_valor,historico_pagamento_em_valor_anteriormente
57,000722117d63c81c,imovel,3.0,12,14,0.749153,0.137698,0.082292
58,000722117d63c81c,mercantil,0.0,5,0,0.000000,0.000000,NaN


----------------------------------------------------------------------------------------------

In [20]:
df_pipe_cluster.head()

,,status_situacao,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_qtd,historico_pagamento_em_valor
id_pessoa,tipo_divida,,,,,
f1e9dde2a3053d51,imovel,3.0,4,0,1.0,1.0
196d80b1b82edbc0,mercantil,0.0,2,2,1.0,1.0
5f21929a71c17a08,imovel,3.0,4,3,1.0,1.0
5f202122440e32a5,mercantil,0.0,2,2,1.0,1.0
cb7ae840eac57e1d,mercantil,3.0,2,2,1.0,1.0


In [21]:
df_pipe_cluster.describe().T

,count,mean,std,min,25%,50%,75%,max
status_situacao,390502.0,1.493821,1.383085,0.0,0.0,2.0,3.000000,3.0
num_dist_cda,390502.0,5.917990,3.490616,2.0,3.0,5.0,8.000000,112.0
quantidade_reparcelamento,390502.0,1.428738,3.608940,0.0,0.0,0.0,1.000000,221.0
historico_pagamento_em_qtd,390502.0,0.241666,0.360382,0.0,0.0,0.0,0.448276,1.0
historico_pagamento_em_valor,390502.0,0.146567,0.272039,0.0,0.0,0.0,0.166122,1.0


# 03) Clusterização dos contribuintes

In [22]:
faixa_n_clusters = [i for i in range(2,16)]
valores_inercia = []
valores_score = []

for k in faixa_n_clusters:
    agrupador = KMeans(n_clusters=k, random_state=1337)
    label = agrupador.fit_predict(df_pipe_cluster)
    print(f"Treinamento do agrupador para K= {k} finalizado")
    
    media_inercia = agrupador.inertia_
    valores_inercia.append(media_inercia)
    print(f"Inércia calculada para o agrupador de K= {k}. Inércia: {media_inercia}")

    media_score = agrupador.score(df_pipe_cluster)
    valores_score.append(media_score)
    print(f"Score calculado para o agrupador de K= {k}. Socre: {media_score}")

Treinamento do agrupador para K= 2 finalizado
Inércia calculada para o agrupador de K= 2. Inércia: 7026270.853602368
Score calculado para o agrupador de K= 2. Socre: -7026270.853602368
Treinamento do agrupador para K= 3 finalizado
Inércia calculada para o agrupador de K= 3. Inércia: 4366954.9156647725
Score calculado para o agrupador de K= 3. Socre: -4366954.9156647725
Treinamento do agrupador para K= 4 finalizado
Inércia calculada para o agrupador de K= 4. Inércia: 3473280.0284419246
Score calculado para o agrupador de K= 4. Socre: -3473280.0284419246
Treinamento do agrupador para K= 5 finalizado
Inércia calculada para o agrupador de K= 5. Inércia: 2907023.1823821464
Score calculado para o agrupador de K= 5. Socre: -2907023.182382147
Treinamento do agrupador para K= 6 finalizado
Inércia calculada para o agrupador de K= 6. Inércia: 2466405.0883437395
Score calculado para o agrupador de K= 6. Socre: -2466405.0883437395
Treinamento do agrupador para K= 7 finalizado
Inércia calculada para

In [23]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = faixa_n_clusters, y = valores_inercia))
fig.update_layout(
    title = "INDICADOR: Inercia para K grupos",
    xaxis_title = "Valores de K",
    yaxis_title = "Inércia",
    font = dict(
        family = "Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

In [24]:
fig =go.Figure()
fig.add_trace(go.Scatter(x = faixa_n_clusters, y = valores_score))
fig.update_layout(
    title="Indicador: Erro quadratico médio para K grupos",
    xaxis_title="Valores de K",
    yaxis_title="MSE",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()

In [26]:
def optimal_number_of_clusters(wcss):
    x1, y1 = 2, wcss[0]
    x2, y2 = 15, wcss[len(wcss)-1]

    distances = []
    for i in range(len(wcss)):
        x0 = i+2
        y0 = wcss[i]
        numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = np.sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(numerator/denominator)
    
    return distances.index(max(distances)) + 2

In [27]:
valor_ideal_k = optimal_number_of_clusters(valores_inercia)

print("Melhor valor de K:", valor_ideal_k)

Melhor valor de K: 5


In [28]:
# Construindo o melhor agrupador de clusteres
VALOR_K = valor_ideal_k

agrupador = KMeans(n_clusters=VALOR_K, random_state=1337)
agrupador.fit_transform(df_pipe_cluster)

# Obtendo o ponto central dos clusteres
centros = agrupador.cluster_centers_
df_centroide = pd.DataFrame(centros, columns = df_pipe_cluster.columns).round(3)
df_centroide['cluster'] = df_centroide.index

# Obtendo o label para cada pessoa
df_pipe_cluster['label_cluster'] = agrupador.labels_

In [29]:
df_pipe_cluster

,,status_situacao,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_qtd,historico_pagamento_em_valor,label_cluster
id_pessoa,tipo_divida,,,,,,
f1e9dde2a3053d51,imovel,3.0,4,0,1.0,1.0,1
196d80b1b82edbc0,mercantil,0.0,2,2,1.0,1.0,1
5f21929a71c17a08,imovel,3.0,4,3,1.0,1.0,1
5f202122440e32a5,mercantil,0.0,2,2,1.0,1.0,1
cb7ae840eac57e1d,mercantil,3.0,2,2,1.0,1.0,1
...,...,...,...,...,...,...,...
6dec5209fbc2f813,imovel,3.0,4,0,0.0,0.0,1
6dec36900dea378c,mercantil,0.0,7,0,0.0,0.0,0
6debc091e4bf4d84,mercantil,0.0,2,0,0.0,0.0,1


In [30]:
df_pipe_cluster.groupby('label_cluster').count()

,status_situacao,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_qtd,historico_pagamento_em_valor
label_cluster,,,,,
0,108634,108634,108634,108634,108634
1,173614,173614,173614,173614,173614
2,33602,33602,33602,33602,33602
3,4793,4793,4793,4793,4793
4,69859,69859,69859,69859,69859


# Centroide

In [31]:
df_centroide

,status_situacao,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_qtd,historico_pagamento_em_valor,cluster
0,0.768,6.601,0.259,0.077,0.041,0
1,1.490,2.998,0.672,0.307,0.212,1
2,2.606,6.629,7.518,0.721,0.367,2
3,2.627,11.115,23.339,0.696,0.238,3
4,2.019,11.414,0.693,0.072,0.034,4


------------------------

# Primeira Divida

In [32]:
# Para ter uma noção de quem seria o centroide
df_primeira_divida = pessoas.query("num_dist_cda == 1")

# Imputando historico_pagamento_em_valor = 1 nos casos que passa de 1
# ESCOLHA DA LARISSA
df_primeira_divida.loc[df_primeira_divida['historico_pagamento_em_valor'] > 1, 'historico_pagamento_em_valor'] = 1
df_primeira_divida

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,valor_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,situacao_ativa,status_situacao,historico_pagamento_em_qtd,historico_pagamento_em_valor
247206,7fcd58cfe066e299,mercantil,1,0,4.0,4.0,533.37,533.37,0.0,0.0,ATIVO,1,1.0,2.0,1.0,1.0
183548,5f225ff188d71bf6,imovel,1,0,12.0,12.0,454.28,454.28,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0
183545,5f21929a71c17a08,mercantil,1,0,1.0,1.0,550.47,550.47,0.0,0.0,BAIXADO,1,0.0,0.0,1.0,1.0
183502,5f1c8f8ff37e7c3e,mercantil,1,0,2.0,2.0,475.05,475.05,0.0,0.0,ATIVO,1,1.0,2.0,1.0,1.0
183488,5f1b78f91fafb2fd,imovel,1,0,5.0,5.0,256.24,256.24,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212539,6deaad1ecbbf645a,imovel,1,0,20.0,0.0,1818.79,0.00,0.0,1.0,ATIVO,1,2.0,3.0,0.0,0.0
212526,6de7ee41718e00fe,imovel,1,0,14.0,0.0,642.70,0.00,0.0,1.0,ATIVO,0,2.0,2.0,0.0,0.0
212545,6deb3707706b25c7,mercantil,1,0,4.0,0.0,1833.70,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0
212546,6deb4b0cba7a76c7,mercantil,1,0,1.0,0.0,321.94,0.00,0.0,0.0,BAIXADO,1,0.0,0.0,0.0,0.0


### Comentários 1ª Dívida

#### 39% inacessível

#### 46% completamente acessível

In [33]:
prim_div_status_sit = df_primeira_divida.groupby('status_situacao')['id_pessoa'].nunique().to_frame().reset_index()
total = prim_div_status_sit['id_pessoa'].sum()
prim_div_status_sit['total'] = total
prim_div_status_sit['perc'] = np.round(prim_div_status_sit['id_pessoa']/total, 5)
prim_div_status_sit

,status_situacao,id_pessoa,total,perc
0,0.0,41253,103464,0.39872
1,1.0,23,103464,0.00022
2,2.0,14065,103464,0.13594
3,3.0,48123,103464,0.46512


In [34]:
data_percentil = {
    'inf': [0, 0.01, 0.25, 0.5, 0.75, 0.99, 1],
    'sup': [0.01, 0.25, 0.5, 0.75, 0.99, 1, 1.01]
}

# Creating a DataFrame from the dictionary
df_cut_percentil = pd.DataFrame(data_percentil)
df_cut_percentil['intervalo'] = df_cut_percentil.apply(paste_intervalo, axis = 1)

# Define the bins and labels for groups
igr_bins = [0, 0.01, 0.25, 0.5, 0.75, 0.99, 1, 1.01]
igr_labels = df_cut_percentil['intervalo']


df_primeira_divida['historico_pagamento_em_qtd_faixas'] = pd.cut(df_primeira_divida['historico_pagamento_em_qtd'],
                                bins = igr_bins,
                                labels = igr_labels,
                                right = False)


df_primeira_divida['historico_pagamento_em_valor_faixas'] = pd.cut(df_primeira_divida['historico_pagamento_em_valor'],
                                bins = igr_bins,
                                labels = igr_labels,
                                right = False)

df_hist_pg_vlr = df_primeira_divida.groupby('historico_pagamento_em_valor_faixas')['historico_pagamento_em_valor_faixas'].count().to_frame()
total = df_hist_pg_vlr['historico_pagamento_em_valor_faixas'].sum()  
df_hist_pg_vlr['total'] = total
df_hist_pg_vlr['perc'] = np.round(df_hist_pg_vlr['historico_pagamento_em_valor_faixas']/total, 4)
df_hist_pg_vlr

,historico_pagamento_em_valor_faixas,total,perc
historico_pagamento_em_valor_faixas,,,
"[0.0, 0.01)",53973,103911,0.5194
"[0.01, 0.25)",7913,103911,0.0762
"[0.25, 0.5)",8612,103911,0.0829
"[0.5, 0.75)",3434,103911,0.0330
"[0.75, 0.99)",217,103911,0.0021
"[0.99, 1.0)",0,103911,0.0000
"[1.0, 1.01)",29762,103911,0.2864


In [35]:
df_hist_pg_qtd = df_primeira_divida.groupby('historico_pagamento_em_qtd_faixas')['historico_pagamento_em_qtd_faixas'].count().to_frame()
total = df_hist_pg_qtd['historico_pagamento_em_qtd_faixas'].sum()  
df_hist_pg_qtd['total'] = total
df_hist_pg_qtd['perc'] = np.round(df_hist_pg_qtd['historico_pagamento_em_qtd_faixas']/total, 4)
df_hist_pg_qtd

,historico_pagamento_em_qtd_faixas,total,perc
historico_pagamento_em_qtd_faixas,,,
"[0.0, 0.01)",53860,103911,0.5183
"[0.01, 0.25)",1313,103911,0.0126
"[0.25, 0.5)",734,103911,0.0071
"[0.5, 0.75)",3007,103911,0.0289
"[0.75, 0.99)",15138,103911,0.1457
"[0.99, 1.0)",99,103911,0.0010
"[1.0, 1.01)",29760,103911,0.2864


In [36]:
df_primeira_divida['status_situacao'].mean()

1.6717286908989424

In [37]:
df_primeira_divida['num_dist_cda'].mean()

1.0

In [38]:
df_primeira_divida['quantidade_reparcelamento'].mean()

0.3565070108073255

In [39]:
df_primeira_divida['historico_pagamento_em_qtd'].mean()

0.43831581990043855

In [40]:
df_primeira_divida['historico_pagamento_em_valor'].mean()

0.34911171131698226

Na totalidade temos +1 cluster especial que é do contribuinte que apareceu a primeira vez em DA

In [41]:
# dicionario_clusteres ={            # status       # qtd   # vlr
#     1: 'PIOR PAGADOR',          C  # 0.006	    0.113	0.045   --->  paga pouquíssimo & tá inacessível     
#     3: 'PAGADOR INTERMEDIARIO', B  # 1.999	    0.177	0.077   --->  paga pouquíssimo & tá menos acessível
#     2: 'BOM PAGADOR',           A  # 3.000 	    0.139	0.067   --->  paga pouquíssimo & tá acessível         
#     0: 'MELHOR PAGADOR',        AA # 2.954	    0.874	0.606   --->  paga bem         & tá acessível	        
#     4: 'PRIMEIRA DIVIDA'           # 1.668        0.437   0.351   --->  paga médio       & tá mais ou menos acessível
# }

# Com K = 4
#status_situacao 	num_dist_cda	quantidade_reparc	historico_pagamento_em_qtd	historico_pagamento_em_valor	cluster
#2.595	            6.761	        7.513	                0.713	                        0.361	                        0
#1.548	            9.855	        0.486	                0.073	                        0.037	                        1
#2.627	            11.095	        23.413	                0.698	                        0.239	                        2
#1.277	            3.601	        0.562	                0.251	                        0.170	                        3

# Com k = 5
#status_situacao num_dist_cda	quantidade_reparc	hist_pg_qtd	   hist_pg_valor   cluster  Num.Contrib
#0.768	         6.601	         0.259	            0.077	        0.041	        0   --- 108.634  # pg mau + nao repar + num médio de CDAs + pouquíssimo acessível [CONTRIB PESSIMO]
#1.490	         2.998	         0.672	            0.307	        0.212	        1   --- 173.614  # pg médio + nao repar + num baixo de CDAs + pouco acessível     [MÉDIO INACESSÍVEL]
#2.606	         6.629	         7.518	            0.721	        0.367	        2   --- 33.602   # pg melhor + repar bem + num médio de CDAs + mt acessível       [CONTRIB EXCEL]
#2.627	         11.115	         23.339	            0.696	        0.238	        3   --- 4.793    # pg bem + repar mt + num alto de CDAs + mt acessível            [CONTRIB NEGOC]
#2.019	         11.414	         0.693	            0.072	        0.034	        4   --- 69.859   # pg mau + nao repar + num alto de CDAs + médio acessível        [RUIM ACESSÍVEL]




# Criar um dicionário com valores escalares
dicionario_clusteres = {
    'class_contribuinte': [0, 1, 2, 3, 4, 5],
    'class_contribuinte_nome': ['CONTRIB PESSIMO', 
                                'MEDIO INACESSIVEL', 
                                'CONTRIB EXCELENTE', 
                                'CONTRIB NEGOCIADOR', 
                                'RUIM ACESSIVEL',
                                'PRIMEIRA DIVIDA']
}

df_dicionario_clusteres = pd.DataFrame(dicionario_clusteres)
df_dicionario_clusteres

,class_contribuinte,class_contribuinte_nome
0,0,CONTRIB PESSIMO
1,1,MEDIO INACESSIVEL
2,2,CONTRIB EXCELENTE
3,3,CONTRIB NEGOCIADOR
4,4,RUIM ACESSIVEL
5,5,PRIMEIRA DIVIDA


In [42]:
df_pipe_cluster.groupby('label_cluster').count()

,status_situacao,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_qtd,historico_pagamento_em_valor
label_cluster,,,,,
0,108634,108634,108634,108634,108634
1,173614,173614,173614,173614,173614
2,33602,33602,33602,33602,33602
3,4793,4793,4793,4793,4793
4,69859,69859,69859,69859,69859


In [43]:
df_centroide

,status_situacao,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_qtd,historico_pagamento_em_valor,cluster
0,0.768,6.601,0.259,0.077,0.041,0
1,1.490,2.998,0.672,0.307,0.212,1
2,2.606,6.629,7.518,0.721,0.367,2
3,2.627,11.115,23.339,0.696,0.238,3
4,2.019,11.414,0.693,0.072,0.034,4


In [44]:
# Salve o DataFrame em um arquivo CSV
if AGRUP_COM_FREQ_PESSOAS == 1:
    df_dicionario_clusteres.to_csv('data/df_dicionario_clusteres_AGRUP_COM_FREQ_PESSOAS.csv', index = False)  
if AGRUP_COM_STATUS_SITUACAO == 1:
    df_dicionario_clusteres.to_csv('data/df_dicionario_clusteres_AGRUP_COM_STATUS_SITUACAO.csv', index = False) 

Classificações dos contribuintes com base no melhor ao pior pagador, baseado no seu histórico e na sua situação atual.

# 04) Cria classificador de class do contribuinte

In [45]:
# Constroi o modelo que prevê qual o grupo do contribuinte

x_cluster = df_pipe_cluster.drop(columns=['label_cluster'])
y_cluster = df_pipe_cluster['label_cluster']

X_train, X_test, y_train, y_test = train_test_split(x_cluster, y_cluster, test_size=0.3, random_state=1337)

In [46]:
model_predict_contribuinte = RandomForestClassifier(random_state=1337)
model_predict_contribuinte.fit(X_train, y_train)

score_validacao = model_predict_contribuinte.score(X_test, y_test)
print("Score de validacao:", score_validacao)

Score de validacao: 0.9998890321038659


In [47]:
# Previsão da classificação para a base total de contribuintes
matriz_previsao_class = pessoas[['status_situacao', 'num_dist_cda', 'quantidade_reparcelamento', 'historico_pagamento_em_qtd', 'historico_pagamento_em_valor']]
pessoas['class_contribuinte'] = model_predict_contribuinte.predict(matriz_previsao_class)

pessoas.loc[pessoas['num_dist_cda'] == 1, 'class_contribuinte'] = 5

In [48]:
#dados_pessoas = dados_pessoas.reset_index()
pessoas

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,valor_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,situacao_ativa,status_situacao,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte
247206,7fcd58cfe066e299,mercantil,1,0,4.0,4.0,533.37,533.37,0.0,0.0,ATIVO,1,1.0,2.0,1.0,1.0,5
467137,f1e9dde2a3053d51,imovel,4,0,79.0,79.0,7987.43,7987.43,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0,1
183548,5f225ff188d71bf6,imovel,1,0,12.0,12.0,454.28,454.28,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0,5
183545,5f21929a71c17a08,mercantil,1,0,1.0,1.0,550.47,550.47,0.0,0.0,BAIXADO,1,0.0,0.0,1.0,1.0,5
49086,196d80b1b82edbc0,mercantil,2,2,8.0,8.0,493.97,493.97,0.0,0.0,SUSPENSO,1,0.0,0.0,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212557,6dec5209fbc2f813,imovel,4,0,136.0,0.0,14185.80,0.00,0.0,1.0,ATIVO,1,2.0,3.0,0.0,0.0,1
212556,6dec36900dea378c,mercantil,7,0,52.0,0.0,19190.68,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,0
212552,6debc091e4bf4d84,mercantil,2,0,6.0,0.0,14930.45,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,1
212551,6debbb4cc485f25f,mercantil,8,0,25.0,0.0,29849.66,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,0


# Exemplo com hash de pessoas

In [49]:
pessoas[pessoas['id_pessoa'] == '000032f6d93a0abd'] 
#status_situacao num_dist_cda	quantidade_reparc	hist_pg_qtd	   hist_pg_valor   cluster  Num.Contrib
#0.768	         6.601	         0.259	            0.077	        0.041	        0   --- 108.634  # pg mau + nao repar + num médio de CDAs + pouquíssimo acessível [CONTRIB PESSIMO]
#1.490	         2.998	         0.672	            0.307	        0.212	        1   --- 173.614  # pg médio + nao repar + num baixo de CDAs + pouco acessível     [MÉDIO INACESSÍVEL]


,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,valor_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,situacao_ativa,status_situacao,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte
1,000032f6d93a0abd,imovel,2,2,10.0,8.0,12715.63,3339.31,0.0,1.0,ATIVO,1,2.0,3.0,0.8,0.262615,1
2,000032f6d93a0abd,mercantil,9,0,66.0,0.0,69091.42,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.000000,0


In [50]:
pessoas[pessoas['id_pessoa'] == 'def2e7cb1e2f6ae1'] 
#status_situacao num_dist_cda	quantidade_reparc	hist_pg_qtd	   hist_pg_valor   cluster  Num.Contrib
#1.490	         2.998	         0.672	            0.307	        0.212	        1   --- 173.614  # pg médio + nao repar + num baixo de CDAs + pouco acessível     [MÉDIO INACESSÍVEL]
#2.606	         6.629	         7.518	            0.721	        0.367	        2   --- 33.602   # pg melhor + repar bem + num médio de CDAs + mt acessível       [CONTRIB EXCEL]


,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,valor_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,situacao_ativa,status_situacao,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte
430200,def2e7cb1e2f6ae1,imovel,7,7,45.0,43.0,4980.41,4137.32,0.0,1.0,ATIVO,1,2.0,3.0,0.955556,0.830719,2
430201,def2e7cb1e2f6ae1,mercantil,2,0,6.0,0.0,33600.12,0.00,0.0,0.0,SUSPENSO,1,0.0,0.0,0.000000,0.000000,1


# Quantos contribuintes tem CDA em imóvel e mercantil?

### 70.354

In [51]:
contrib_num_tipo_divida = dados_pessoas.groupby('id_pessoa')['tipo_divida'].nunique().to_frame().reset_index()
contrib_num_tipo_divida = contrib_num_tipo_divida.sort_values(by = 'tipo_divida', ascending = False)
contrib_02_tipo_divida = contrib_num_tipo_divida[contrib_num_tipo_divida['tipo_divida'] > 1]
contrib_02_tipo_divida = contrib_02_tipo_divida.drop(columns = 'tipo_divida')

In [52]:
dados_pessoas_aux = dados_pessoas[["tipo_divida", "cda", "id_pessoa", "valor_tot"]]
contrib_02_tipo_divida = pd.merge(dados_pessoas_aux, contrib_02_tipo_divida, how = "right", on = "id_pessoa")
contrib_02_tipo_divida = contrib_02_tipo_divida.sort_values(by = "id_pessoa")
contrib_02_tipo_divida

,tipo_divida,cda,id_pessoa,valor_tot
579016,mercantil,fcab2f9c51259d2ed8a3e5cebc255ca62,000032f6d93a0abd,1789.80
579015,mercantil,db950944ce8b2b706b3a8e268c80aedf2,000032f6d93a0abd,22147.06
579014,mercantil,bfb98e5cd3a4eb2e7c8fa7a67092eaab2,000032f6d93a0abd,7519.63
579013,mercantil,b269b3c8a08334b29fcacff532cf50c42,000032f6d93a0abd,1897.32
579012,mercantil,b208dd365dbb6b4c6b3a8e268c80aedf2,000032f6d93a0abd,9769.89
...,...,...,...,...
687897,imovel,858bdcca9dce6e914bd01ecb8277da0b1,fffeb78fd42d2624,1015.13
687898,imovel,996bf4fc3cbc77229ceed47545e621dd1,fffeb78fd42d2624,4162.30
687900,mercantil,bb2daefbbfd5c13fe52281a70553db462,fffeb78fd42d2624,885.21
687902,imovel,fc3f9d49392410942b46f673bf0c61251,fffeb78fd42d2624,11056.72


In [53]:
contrib_02_tipo_divida['id_pessoa'].nunique() /494413

0.14229803827973778

In [54]:
contrib_02_tipo_divida_metrics = contrib_02_tipo_divida.groupby(['id_pessoa', 'tipo_divida']).agg({
    'cda': ['nunique'], 
    'valor_tot': ['sum']    
})
contrib_02_tipo_divida_metrics = contrib_02_tipo_divida_metrics.reset_index()
contrib_02_tipo_divida_metrics

,id_pessoa,tipo_divida,cda,valor_tot
,,,nunique,sum
0,000032f6d93a0abd,imovel,2,12715.63
1,000032f6d93a0abd,mercantil,9,69091.42
2,0000e13a7d8d7537,imovel,9,12683.16
3,0000e13a7d8d7537,mercantil,5,8306.33
4,0001cc5c538504ce,imovel,6,13948.46
...,...,...,...,...
140703,fffc4c97f1f6c954,mercantil,2,1651.71
140704,fffe4c8195b51bd9,imovel,2,1060.03
140705,fffe4c8195b51bd9,mercantil,1,1833.70


In [55]:
# Salve o DataFrame em um arquivo CSV
if AGRUP_COM_FREQ_PESSOAS == 1:
    contrib_02_tipo_divida_metrics.to_csv('data/contrib_02_tipo_divida_metrics_AGRUP_COM_FREQ_PESSOAS.csv', index = False)  

In [56]:
# Vendo as dívidas desses pessoas
contrib_02_tipo_divida_aux = contrib_02_tipo_divida[['tipo_divida',	'id_pessoa']]
contrib_02_tipo_divida_pessoas = pd.merge(contrib_02_tipo_divida_aux, pessoas, on = ['tipo_divida', 'id_pessoa'], how = 'left')
contrib_02_tipo_divida_pessoas = contrib_02_tipo_divida_pessoas.drop_duplicates()
contrib_02_tipo_divida_pessoas

,tipo_divida,id_pessoa,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,valor_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,situacao_ativa,status_situacao,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte
0,mercantil,000032f6d93a0abd,9,0,66.0,0.0,69091.42,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.000000,0.000000,0
9,imovel,000032f6d93a0abd,2,2,10.0,8.0,12715.63,3339.31,0.0,1.0,ATIVO,1,2.0,3.0,0.800000,0.262615,1
11,mercantil,0000e13a7d8d7537,5,6,42.0,27.0,8306.33,1980.75,76.0,0.0,ATIVO,1,2.0,3.0,0.642857,0.238463,2
12,imovel,0000e13a7d8d7537,9,0,177.0,0.0,12683.16,0.00,76.0,1.0,ATIVO,0,2.0,2.0,0.000000,0.000000,4
25,imovel,0001cc5c538504ce,6,0,103.0,0.0,13948.46,0.00,0.0,1.0,ATIVO,1,2.0,3.0,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690520,mercantil,fffc4c97f1f6c954,2,1,7.0,2.0,1651.71,46.47,0.0,0.0,BAIXADO,1,0.0,0.0,0.285714,0.028134,1
690530,imovel,fffe4c8195b51bd9,2,0,32.0,32.0,1060.03,1060.03,0.0,1.0,ATIVO,1,2.0,3.0,1.000000,1.000000,1
690531,mercantil,fffe4c8195b51bd9,1,0,4.0,0.0,1833.70,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.000000,0.000000,5
690533,mercantil,fffeb78fd42d2624,3,0,15.0,0.0,13050.71,0.00,0.0,0.0,SUSPENSO,1,0.0,0.0,0.000000,0.000000,1


# Voltando para os agrupamentos

In [57]:
# Nomeando a classificação com label de prioridade

pessoas = pd.merge(pessoas, 
         df_dicionario_clusteres, 
         on = "class_contribuinte",
         how = "left")

pessoas

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,valor_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,situacao_ativa,status_situacao,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte,class_contribuinte_nome
0,7fcd58cfe066e299,mercantil,1,0,4.0,4.0,533.37,533.37,0.0,0.0,ATIVO,1,1.0,2.0,1.0,1.0,5,PRIMEIRA DIVIDA
1,f1e9dde2a3053d51,imovel,4,0,79.0,79.0,7987.43,7987.43,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0,1,MEDIO INACESSIVEL
2,5f225ff188d71bf6,imovel,1,0,12.0,12.0,454.28,454.28,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0,5,PRIMEIRA DIVIDA
3,5f21929a71c17a08,mercantil,1,0,1.0,1.0,550.47,550.47,0.0,0.0,BAIXADO,1,0.0,0.0,1.0,1.0,5,PRIMEIRA DIVIDA
4,196d80b1b82edbc0,mercantil,2,2,8.0,8.0,493.97,493.97,0.0,0.0,SUSPENSO,1,0.0,0.0,1.0,1.0,1,MEDIO INACESSIVEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494408,6dec5209fbc2f813,imovel,4,0,136.0,0.0,14185.80,0.00,0.0,1.0,ATIVO,1,2.0,3.0,0.0,0.0,1,MEDIO INACESSIVEL
494409,6dec36900dea378c,mercantil,7,0,52.0,0.0,19190.68,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,0,CONTRIB PESSIMO
494410,6debc091e4bf4d84,mercantil,2,0,6.0,0.0,14930.45,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,1,MEDIO INACESSIVEL
494411,6debbb4cc485f25f,mercantil,8,0,25.0,0.0,29849.66,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,0,CONTRIB PESSIMO


In [58]:
pessoas.columns

Index(['id_pessoa', 'tipo_divida', 'num_dist_cda', 'quantidade_reparcelamento',
       'deb_totais', 'deb_pagos', 'valor_tot', 'valor_pago', 'qtd_notas_2anos',
       'edificacao', 'situacao', 'cpf_cnpj_existe', 'situacao_ativa',
       'status_situacao', 'historico_pagamento_em_qtd',
       'historico_pagamento_em_valor', 'class_contribuinte',
       'class_contribuinte_nome'],
      dtype='object')

In [59]:
# Incluindo variável 'tipo_divida'
df_classificao_contribuinte = pessoas[['id_pessoa', 'tipo_divida','class_contribuinte_nome']]
df_classificao_contribuinte

,id_pessoa,tipo_divida,class_contribuinte_nome
0,7fcd58cfe066e299,mercantil,PRIMEIRA DIVIDA
1,f1e9dde2a3053d51,imovel,MEDIO INACESSIVEL
2,5f225ff188d71bf6,imovel,PRIMEIRA DIVIDA
3,5f21929a71c17a08,mercantil,PRIMEIRA DIVIDA
4,196d80b1b82edbc0,mercantil,MEDIO INACESSIVEL
...,...,...,...
494408,6dec5209fbc2f813,imovel,MEDIO INACESSIVEL
494409,6dec36900dea378c,mercantil,CONTRIB PESSIMO
494410,6debc091e4bf4d84,mercantil,MEDIO INACESSIVEL
494411,6debbb4cc485f25f,mercantil,CONTRIB PESSIMO


In [60]:
df_classificao_contribuinte.groupby('class_contribuinte_nome').count()

,id_pessoa,tipo_divida
class_contribuinte_nome,,
CONTRIB EXCELENTE,33610,33610
CONTRIB NEGOCIADOR,4794,4794
CONTRIB PESSIMO,108633,108633
MEDIO INACESSIVEL,173614,173614
PRIMEIRA DIVIDA,103911,103911
RUIM ACESSIVEL,69851,69851


In [61]:
base_conjunta.columns

Index(['cda', 'tipo_divida', 'id_pessoa', 'atividade_principal', 'situacao',
       'tipo_tributo', 'valor_pago', 'valor_tot', 'vlr_tributo', 'vlr_taxa',
       'competencia_divida', 'inscricao_divida', 'arrecadacao_divida',
       'ajuizamento_divida', 'edificacao', 'cpf_cnpj_existe', 'protesto',
       'ajuizamento', 'refis', 'deb_totais', 'deb_pagos', 'anos_idade_da',
       'quantidade_reparcelamento'],
      dtype='object')

# 05) Análise discriminante da classificação do contribuintes para o % Pago das dívidas de cada CDA

In [62]:
# Calcula variável target y
base_conjunta.loc[(base_conjunta['valor_pago'].isna()) | (base_conjunta['valor_pago'] == 0) , 'valor_pago'] = 1
base_conjunta.loc[(base_conjunta['valor_tot'].isna()) | (base_conjunta['valor_tot'] == 0) , 'valor_tot'] = 1

base_conjunta['percentual_pago_cda'] = base_conjunta['valor_pago'] / base_conjunta['valor_tot']

In [63]:
# Imputando historico_pagamento_em_valor = 1 nos casos que passa de 1
# ESCOLHA DA LARISSA
base_conjunta.loc[base_conjunta['percentual_pago_cda'] > 1, 'percentual_pago_cda'] = 1

In [64]:
df_classificao_contribuinte

,id_pessoa,tipo_divida,class_contribuinte_nome
0,7fcd58cfe066e299,mercantil,PRIMEIRA DIVIDA
1,f1e9dde2a3053d51,imovel,MEDIO INACESSIVEL
2,5f225ff188d71bf6,imovel,PRIMEIRA DIVIDA
3,5f21929a71c17a08,mercantil,PRIMEIRA DIVIDA
4,196d80b1b82edbc0,mercantil,MEDIO INACESSIVEL
...,...,...,...
494408,6dec5209fbc2f813,imovel,MEDIO INACESSIVEL
494409,6dec36900dea378c,mercantil,CONTRIB PESSIMO
494410,6debc091e4bf4d84,mercantil,MEDIO INACESSIVEL
494411,6debbb4cc485f25f,mercantil,CONTRIB PESSIMO


In [65]:
# Separação dos dados para a análise discriminante dos grupos de contribuintes
df_analise_discriminante = pd.merge(
    left = base_conjunta, on=['id_pessoa', 'tipo_divida'], right=df_classificao_contribuinte, how='left'
)

df_analise_discriminante = df_analise_discriminante[['id_pessoa', 'tipo_divida', 'percentual_pago_cda', 'class_contribuinte_nome']]

In [66]:
base_conjunta

,cda,tipo_divida,id_pessoa,atividade_principal,situacao,tipo_tributo,valor_pago,valor_tot,vlr_tributo,vlr_taxa,...,edificacao,cpf_cnpj_existe,protesto,ajuizamento,refis,deb_totais,deb_pagos,anos_idade_da,quantidade_reparcelamento,percentual_pago_cda
0,00000123c3d4731c6b3a8e268c80aedf1,imovel,ac28642d7c82b33f,APARTAMENTO,ATIVO,IPTU,1.0,2048.73,762.08,1286.65,...,1,1,0,0,0,40.0,0.0,1.0,0,0.000488
1,00000123c3d4731c6b3a8e268c80aedf2,mercantil,fc4b99b807fbed41,ATIVIDADES DE TELEATENDIMENTO,SUSPENSO,ISS,1.0,2515.85,0.00,2515.85,...,0,1,0,0,0,3.0,0.0,1.0,0,0.000397
2,00000b44c5ba1e669ceed47545e621dd2,mercantil,96e8e553de69d7a4,COMERCIO VAREJISTA DE ARTIGOS DE ARMARINHO,INAPTO,ISS,1.0,1278.97,0.00,1278.97,...,0,1,0,0,0,4.0,0.0,21.0,0,0.000782
3,000014e359592e62d8a3e5cebc255ca6E,mercantil,6dbe14da38a31dc1,Comercio varejista especializado de equipament...,ATIVO,ISS,1.0,847.02,847.02,0.00,...,0,1,0,0,0,1.0,0.0,0.0,0,0.001181
4,0000331f601a73e52b46f673bf0c61251,imovel,870c08c252b25ad1,APARTAMENTO,ATIVO,IPTU,1.0,344.12,147.20,196.92,...,1,1,0,0,0,6.0,0.0,11.0,0,0.002906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2414893,ffffd8af5760cada9fcacff532cf50c41,imovel,d411f54ad97d1f39,NaN,ATIVO,IPTU,1.0,1728.13,790.85,937.28,...,0,0,0,0,0,19.0,0.0,4.0,0,0.000579
2414894,ffffdb721b90282718565e26094f06ee2,mercantil,51d6043a4efa1ed1,PUBLICITARIO (NÍVEL MÉDIO),ATIVO,ISS,1.0,240.77,240.77,0.00,...,0,1,0,0,0,1.0,0.0,6.0,0,0.004153
2414895,ffffdc78aa7a90e26b3a8e268c80aedf1,imovel,9b512f210b460120,APARTAMENTO,ATIVO,IPTU,1.0,9390.43,5113.61,4276.82,...,1,1,0,1,0,30.0,0.0,25.0,0,0.000106
2414896,fffff192f894983c9ceed47545e621dd1,imovel,dd5538d2d38803a7,CASA,ATIVO,IPTU,1.0,16594.80,11374.98,5219.82,...,1,1,0,1,0,32.0,0.0,15.0,0,0.000060


In [67]:
df_analise_discriminante = df_analise_discriminante.set_index(['id_pessoa', 'tipo_divida'])
df_analise_discriminante

,,percentual_pago_cda,class_contribuinte_nome
id_pessoa,tipo_divida,,
ac28642d7c82b33f,imovel,0.000488,MEDIO INACESSIVEL
fc4b99b807fbed41,mercantil,0.000397,CONTRIB PESSIMO
96e8e553de69d7a4,mercantil,0.000782,MEDIO INACESSIVEL
6dbe14da38a31dc1,mercantil,0.001181,CONTRIB EXCELENTE
870c08c252b25ad1,imovel,0.002906,RUIM ACESSIVEL
...,...,...,...
d411f54ad97d1f39,imovel,0.000579,RUIM ACESSIVEL
51d6043a4efa1ed1,mercantil,0.004153,MEDIO INACESSIVEL
9b512f210b460120,imovel,0.000106,CONTRIB EXCELENTE


In [68]:
df_analise_discriminante = df_analise_discriminante.reset_index()#.groupby("class_contribuinte_nome").describe().T
df_analise_discriminante

,id_pessoa,tipo_divida,percentual_pago_cda,class_contribuinte_nome
0,ac28642d7c82b33f,imovel,0.000488,MEDIO INACESSIVEL
1,fc4b99b807fbed41,mercantil,0.000397,CONTRIB PESSIMO
2,96e8e553de69d7a4,mercantil,0.000782,MEDIO INACESSIVEL
3,6dbe14da38a31dc1,mercantil,0.001181,CONTRIB EXCELENTE
4,870c08c252b25ad1,imovel,0.002906,RUIM ACESSIVEL
...,...,...,...,...
2414893,d411f54ad97d1f39,imovel,0.000579,RUIM ACESSIVEL
2414894,51d6043a4efa1ed1,mercantil,0.004153,MEDIO INACESSIVEL
2414895,9b512f210b460120,imovel,0.000106,CONTRIB EXCELENTE
2414896,dd5538d2d38803a7,imovel,0.000060,CONTRIB EXCELENTE


In [69]:
# Dummyzando a variável de classificação 

ohe = OneHotEncoder(dtype=int)

colunas_ohe = ohe.fit_transform(df_analise_discriminante[['class_contribuinte_nome']]).toarray()
df_2 = pd.DataFrame(colunas_ohe, columns=ohe.get_feature_names_out(['class_contribuinte_nome']))
df_2 

,class_contribuinte_nome_CONTRIB EXCELENTE,class_contribuinte_nome_CONTRIB NEGOCIADOR,class_contribuinte_nome_CONTRIB PESSIMO,class_contribuinte_nome_MEDIO INACESSIVEL,class_contribuinte_nome_PRIMEIRA DIVIDA,class_contribuinte_nome_RUIM ACESSIVEL
0,0,0,0,1,0,0
1,0,0,1,0,0,0
2,0,0,0,1,0,0
3,1,0,0,0,0,0
4,0,0,0,0,0,1
...,...,...,...,...,...,...
2414893,0,0,0,0,0,1
2414894,0,0,0,1,0,0
2414895,1,0,0,0,0,0
2414896,1,0,0,0,0,0


In [70]:
df_n_categorico = df_analise_discriminante.drop(columns=['class_contribuinte_nome'], axis=1)
df_n_categorico

,id_pessoa,tipo_divida,percentual_pago_cda
0,ac28642d7c82b33f,imovel,0.000488
1,fc4b99b807fbed41,mercantil,0.000397
2,96e8e553de69d7a4,mercantil,0.000782
3,6dbe14da38a31dc1,mercantil,0.001181
4,870c08c252b25ad1,imovel,0.002906
...,...,...,...
2414893,d411f54ad97d1f39,imovel,0.000579
2414894,51d6043a4efa1ed1,mercantil,0.004153
2414895,9b512f210b460120,imovel,0.000106
2414896,dd5538d2d38803a7,imovel,0.000060


In [71]:
df_pipe_discriminante = pd.concat([df_n_categorico, df_2], axis=1)

In [72]:
df_pipe_discriminante = df_pipe_discriminante.set_index(['id_pessoa', 'tipo_divida'])
df_pipe_discriminante

,,percentual_pago_cda,class_contribuinte_nome_CONTRIB EXCELENTE,class_contribuinte_nome_CONTRIB NEGOCIADOR,class_contribuinte_nome_CONTRIB PESSIMO,class_contribuinte_nome_MEDIO INACESSIVEL,class_contribuinte_nome_PRIMEIRA DIVIDA,class_contribuinte_nome_RUIM ACESSIVEL
id_pessoa,tipo_divida,,,,,,,
ac28642d7c82b33f,imovel,0.000488,0,0,0,1,0,0
fc4b99b807fbed41,mercantil,0.000397,0,0,1,0,0,0
96e8e553de69d7a4,mercantil,0.000782,0,0,0,1,0,0
6dbe14da38a31dc1,mercantil,0.001181,1,0,0,0,0,0
870c08c252b25ad1,imovel,0.002906,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
d411f54ad97d1f39,imovel,0.000579,0,0,0,0,0,1
51d6043a4efa1ed1,mercantil,0.004153,0,0,0,1,0,0
9b512f210b460120,imovel,0.000106,1,0,0,0,0,0


In [73]:
x_analise_discriminante = df_pipe_discriminante.drop(columns=['percentual_pago_cda'])
y_analise_discriminante = df_pipe_discriminante['percentual_pago_cda'].astype('int')

In [74]:
analise_discriminante = LinearDiscriminantAnalysis()
analise_discriminante.fit(x_analise_discriminante, y_analise_discriminante)

LinearDiscriminantAnalysis()

In [75]:
dados_analise_disc = {'variavel': analise_discriminante.feature_names_in_, 'coeficiente' : analise_discriminante.coef_[0].round(5)}
pesos_analise_disc = pd.DataFrame(dados_analise_disc).sort_values('variavel').reset_index().drop(columns=['index'])

In [76]:
pesos_analise_disc

,variavel,coeficiente
0,class_contribuinte_nome_CONTRIB EXCELENTE,1.53379
1,class_contribuinte_nome_CONTRIB NEGOCIADOR,0.18144
2,class_contribuinte_nome_CONTRIB PESSIMO,-0.79172
3,class_contribuinte_nome_MEDIO INACESSIVEL,0.84104
4,class_contribuinte_nome_PRIMEIRA DIVIDA,2.14301
5,class_contribuinte_nome_RUIM ACESSIVEL,-0.89373


In [77]:
pesos_analise_disc['class_contribuinte_nome'] = pesos_analise_disc['variavel'].str.replace('class_contribuinte_nome_', '')
pesos_analise_disc

,variavel,coeficiente,class_contribuinte_nome
0,class_contribuinte_nome_CONTRIB EXCELENTE,1.53379,CONTRIB EXCELENTE
1,class_contribuinte_nome_CONTRIB NEGOCIADOR,0.18144,CONTRIB NEGOCIADOR
2,class_contribuinte_nome_CONTRIB PESSIMO,-0.79172,CONTRIB PESSIMO
3,class_contribuinte_nome_MEDIO INACESSIVEL,0.84104,MEDIO INACESSIVEL
4,class_contribuinte_nome_PRIMEIRA DIVIDA,2.14301,PRIMEIRA DIVIDA
5,class_contribuinte_nome_RUIM ACESSIVEL,-0.89373,RUIM ACESSIVEL


In [78]:
# Salve o DataFrame em um arquivo CSV
if AGRUP_COM_FREQ_PESSOAS == 1:
    pesos_analise_disc.to_csv('data/pesos_analise_disc_AGRUP_COM_FREQ_PESSOAS.csv', index = False)  
if AGRUP_COM_STATUS_SITUACAO == 1:
    pesos_analise_disc.to_csv('data/pesos_analise_disc_AGRUP_COM_STATUS_SITUACAO.csv', index = False) 

# 06) Salva o modelo de classificação dos contribuintes

In [79]:
def salva_modelo_serializado(nome_modelo_serializado, modelo):
    sav_best_model = open(nome_modelo_serializado, 'wb')
    pickle.dump(modelo, sav_best_model)
    sav_best_model.close()

    pathModelo = modelsPath+"\\"+os.path.join(nome_modelo_serializado)
    shutil.move(os.path.abspath(nome_modelo_serializado), pathModelo)

In [80]:
if AGRUP_COM_FREQ_PESSOAS == 1:
    salva_modelo_serializado("classificador-contribuinte_prime.pkl", model_predict_contribuinte)
if AGRUP_COM_STATUS_SITUACAO == 1:
    salva_modelo_serializado("classificador-contribuinte-AGRUP_COM_STATUS_SITUACAO_v4.pkl", model_predict_contribuinte)

# SALVAR PESOS

In [81]:
# Use o método str.replace() para substituir o caractere '#' por uma string vazia na coluna 'texto'
pesos_analise_disc['class_contribuinte_nome'] = pesos_analise_disc['variavel'].str.replace('class_contribuinte_nome_', '')
pesos_analise_disc 

,variavel,coeficiente,class_contribuinte_nome
0,class_contribuinte_nome_CONTRIB EXCELENTE,1.53379,CONTRIB EXCELENTE
1,class_contribuinte_nome_CONTRIB NEGOCIADOR,0.18144,CONTRIB NEGOCIADOR
2,class_contribuinte_nome_CONTRIB PESSIMO,-0.79172,CONTRIB PESSIMO
3,class_contribuinte_nome_MEDIO INACESSIVEL,0.84104,MEDIO INACESSIVEL
4,class_contribuinte_nome_PRIMEIRA DIVIDA,2.14301,PRIMEIRA DIVIDA
5,class_contribuinte_nome_RUIM ACESSIVEL,-0.89373,RUIM ACESSIVEL


In [82]:
pesos_analise_disc = pd.merge(pesos_analise_disc, 
         df_dicionario_clusteres, 
         on = "class_contribuinte_nome",
         how = "left")

pesos_analise_disc  

,variavel,coeficiente,class_contribuinte_nome,class_contribuinte
0,class_contribuinte_nome_CONTRIB EXCELENTE,1.53379,CONTRIB EXCELENTE,2
1,class_contribuinte_nome_CONTRIB NEGOCIADOR,0.18144,CONTRIB NEGOCIADOR,3
2,class_contribuinte_nome_CONTRIB PESSIMO,-0.79172,CONTRIB PESSIMO,0
3,class_contribuinte_nome_MEDIO INACESSIVEL,0.84104,MEDIO INACESSIVEL,1
4,class_contribuinte_nome_PRIMEIRA DIVIDA,2.14301,PRIMEIRA DIVIDA,5
5,class_contribuinte_nome_RUIM ACESSIVEL,-0.89373,RUIM ACESSIVEL,4


In [83]:
# Salve o DataFrame em um arquivo CSV
if AGRUP_COM_FREQ_PESSOAS == 1:
    pesos_analise_disc.to_csv('data/pesos_analise_disc_AGRUP_COM_FREQ_PESSOAS.csv', index = False)  
if AGRUP_COM_STATUS_SITUACAO == 1:
    pesos_analise_disc.to_csv('data/pesos_analise_disc_AGRUP_COM_STATUS_SITUACAO.csv', index = False)  

In [84]:
df_pipe_cluster

,,status_situacao,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_qtd,historico_pagamento_em_valor,label_cluster
id_pessoa,tipo_divida,,,,,,
f1e9dde2a3053d51,imovel,3.0,4,0,1.0,1.0,1
196d80b1b82edbc0,mercantil,0.0,2,2,1.0,1.0,1
5f21929a71c17a08,imovel,3.0,4,3,1.0,1.0,1
5f202122440e32a5,mercantil,0.0,2,2,1.0,1.0,1
cb7ae840eac57e1d,mercantil,3.0,2,2,1.0,1.0,1
...,...,...,...,...,...,...,...
6dec5209fbc2f813,imovel,3.0,4,0,0.0,0.0,1
6dec36900dea378c,mercantil,0.0,7,0,0.0,0.0,0
6debc091e4bf4d84,mercantil,0.0,2,0,0.0,0.0,1


# ANALISANDO

In [85]:
# df_pipe_cluster2 = df_pipe_cluster.reset_index()
# df = df_pipe_cluster2.groupby('label_cluster')['id_pessoa'].nunique().to_frame().reset_index()
# total = df['id_pessoa'].sum()
# df['perc'] = df['id_pessoa']/total
# df

,label_cluster,id_pessoa,perc
0,0,106011,0.280301
1,1,165172,0.436727
2,2,33373,0.088241
3,3,4786,0.012655
4,4,68862,0.182076


In [87]:
#dados_pessoas.reset_index(inplace = True)

# Distribuição dos Contribuintes

In [89]:
freq_classecontrib = pessoas.groupby(['class_contribuinte_nome'])['id_pessoa'].nunique().to_frame().reset_index()
tot = freq_classecontrib['id_pessoa'].sum()
freq_classecontrib['perc'] = np.round(freq_classecontrib['id_pessoa']/tot, 4)
freq_classecontrib

,class_contribuinte_nome,id_pessoa,perc
0,CONTRIB EXCELENTE,33381,0.0697
1,CONTRIB NEGOCIADOR,4787,0.0100
2,CONTRIB PESSIMO,106010,0.2214
3,MEDIO INACESSIVEL,165172,0.3450
4,PRIMEIRA DIVIDA,100601,0.2101
5,RUIM ACESSIVEL,68854,0.1438


In [90]:
# dict = {"class_contribuinte_nome": ['PRIMEIRA DIVIDA', 'MELHOR PAGADOR', 
#                                     'BOM PAGADOR', 'PAGADOR INTERMEDIARIO', 'PIOR PAGADOR'],
#         "ordem_contrib": [0, 1, 2 ,3, 4]}

# df_ordenar_class_contrib = pd.DataFrame(dict)
# df_ordenar_class_contrib

dict = {"class_contribuinte_nome": ['CONTRIB EXCELENTE', 'CONTRIB NEGOCIADOR', 
                                    'MEDIO INACESSIVEL', 'RUIM ACESSIVEL', 'CONTRIB PESSIMO', 'PRIMEIRA DIVIDA'],
         "ordem_contrib": [0, 1, 2 ,3, 4, 5]}

df_ordenar_class_contrib = pd.DataFrame(dict)
df_ordenar_class_contrib

,class_contribuinte_nome,ordem_contrib
0,CONTRIB EXCELENTE,0
1,CONTRIB NEGOCIADOR,1
2,MEDIO INACESSIVEL,2
3,RUIM ACESSIVEL,3
4,CONTRIB PESSIMO,4
5,PRIMEIRA DIVIDA,5


In [ ]:
# data_ord_rat_div = {
#     'ord_rat_div': [0, 1, 2, 3],
#     'rating_divida': ['ALTISSIMA', 'ALTA', 'MEDIA', 'BAIXISSIMA'],
#     'rating_divida_label_ord': ['00_ALTISSIMA', '01_ALTA', '02_MEDIA', '03_BAIXISSIMA']
# }

# df_ord_rat_div = pd.DataFrame(data_ord_rat_div)

In [91]:
freq_classecontrib = pd.merge(
    left = freq_classecontrib, 
    right = df_ordenar_class_contrib,
    on = 'class_contribuinte_nome',
    how = "left"
)

freq_classecontrib = freq_classecontrib.sort_values(by = ['ordem_contrib'])
total = freq_classecontrib['id_pessoa'].sum()
freq_classecontrib['total'] = total
freq_classecontrib

,class_contribuinte_nome,id_pessoa,perc,ordem_contrib,total
0,CONTRIB EXCELENTE,33381,0.0697,0,478805
1,CONTRIB NEGOCIADOR,4787,0.0100,1,478805
3,MEDIO INACESSIVEL,165172,0.3450,2,478805
5,RUIM ACESSIVEL,68854,0.1438,3,478805
2,CONTRIB PESSIMO,106010,0.2214,4,478805
4,PRIMEIRA DIVIDA,100601,0.2101,5,478805


In [92]:
# Salve o DataFrame em um arquivo CSV
if AGRUP_COM_FREQ_PESSOAS == 1:
    freq_classecontrib.to_csv('data/distribuicao_contribuintes_AGRUP_COM_FREQ_PESSOAS.csv', index = False)  
if AGRUP_COM_STATUS_SITUACAO == 1:
    freq_classecontrib.to_csv('data/distribuicao_contribuintes_AGRUP_COM_STATUS_SITUACAO.csv', index = False)  

# Previsão

In [93]:
matriz_previsao_class = pessoas[['status_situacao', 'num_dist_cda', 'quantidade_reparcelamento', 'historico_pagamento_em_qtd', 'historico_pagamento_em_valor']]
matriz_previsao_class

,status_situacao,num_dist_cda,quantidade_reparcelamento,historico_pagamento_em_qtd,historico_pagamento_em_valor
0,2.0,1,0,1.0,1.0
1,3.0,4,0,1.0,1.0
2,3.0,1,0,1.0,1.0
3,0.0,1,0,1.0,1.0
4,0.0,2,2,1.0,1.0
...,...,...,...,...,...
494408,3.0,4,0,0.0,0.0
494409,0.0,7,0,0.0,0.0
494410,0.0,2,0,0.0,0.0
494411,0.0,8,0,0.0,0.0


In [94]:
pessoas['class_contribuinte'] = model_predict_contribuinte.predict(matriz_previsao_class)
pessoas

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,valor_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,situacao_ativa,status_situacao,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte,class_contribuinte_nome
0,7fcd58cfe066e299,mercantil,1,0,4.0,4.0,533.37,533.37,0.0,0.0,ATIVO,1,1.0,2.0,1.0,1.0,1,PRIMEIRA DIVIDA
1,f1e9dde2a3053d51,imovel,4,0,79.0,79.0,7987.43,7987.43,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0,1,MEDIO INACESSIVEL
2,5f225ff188d71bf6,imovel,1,0,12.0,12.0,454.28,454.28,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0,1,PRIMEIRA DIVIDA
3,5f21929a71c17a08,mercantil,1,0,1.0,1.0,550.47,550.47,0.0,0.0,BAIXADO,1,0.0,0.0,1.0,1.0,1,PRIMEIRA DIVIDA
4,196d80b1b82edbc0,mercantil,2,2,8.0,8.0,493.97,493.97,0.0,0.0,SUSPENSO,1,0.0,0.0,1.0,1.0,1,MEDIO INACESSIVEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494408,6dec5209fbc2f813,imovel,4,0,136.0,0.0,14185.80,0.00,0.0,1.0,ATIVO,1,2.0,3.0,0.0,0.0,1,MEDIO INACESSIVEL
494409,6dec36900dea378c,mercantil,7,0,52.0,0.0,19190.68,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,0,CONTRIB PESSIMO
494410,6debc091e4bf4d84,mercantil,2,0,6.0,0.0,14930.45,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,1,MEDIO INACESSIVEL
494411,6debbb4cc485f25f,mercantil,8,0,25.0,0.0,29849.66,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,0,CONTRIB PESSIMO


In [95]:
pessoas.loc[pessoas['num_dist_cda'] == 1, 'class_contribuinte'] = 5
pessoas

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,valor_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,situacao_ativa,status_situacao,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte,class_contribuinte_nome
0,7fcd58cfe066e299,mercantil,1,0,4.0,4.0,533.37,533.37,0.0,0.0,ATIVO,1,1.0,2.0,1.0,1.0,5,PRIMEIRA DIVIDA
1,f1e9dde2a3053d51,imovel,4,0,79.0,79.0,7987.43,7987.43,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0,1,MEDIO INACESSIVEL
2,5f225ff188d71bf6,imovel,1,0,12.0,12.0,454.28,454.28,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0,5,PRIMEIRA DIVIDA
3,5f21929a71c17a08,mercantil,1,0,1.0,1.0,550.47,550.47,0.0,0.0,BAIXADO,1,0.0,0.0,1.0,1.0,5,PRIMEIRA DIVIDA
4,196d80b1b82edbc0,mercantil,2,2,8.0,8.0,493.97,493.97,0.0,0.0,SUSPENSO,1,0.0,0.0,1.0,1.0,1,MEDIO INACESSIVEL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494408,6dec5209fbc2f813,imovel,4,0,136.0,0.0,14185.80,0.00,0.0,1.0,ATIVO,1,2.0,3.0,0.0,0.0,1,MEDIO INACESSIVEL
494409,6dec36900dea378c,mercantil,7,0,52.0,0.0,19190.68,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,0,CONTRIB PESSIMO
494410,6debc091e4bf4d84,mercantil,2,0,6.0,0.0,14930.45,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,1,MEDIO INACESSIVEL
494411,6debbb4cc485f25f,mercantil,8,0,25.0,0.0,29849.66,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,0,CONTRIB PESSIMO


# Pesos dos Contribuintes

In [96]:
pesos_analise_disc_aux = pesos_analise_disc[["coeficiente",	"class_contribuinte_nome",	"class_contribuinte"]]

In [97]:
# Nomeando a classificação com label de prioridade
pessoas = pd.merge(pessoas, 
         pesos_analise_disc_aux, 
         on = ["class_contribuinte", "class_contribuinte_nome"],
         how = "left")

pessoas = pessoas.rename(columns = {'coeficiente':'class_contribuinte_peso'})
pessoas

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,valor_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,situacao_ativa,status_situacao,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte,class_contribuinte_nome,class_contribuinte_peso
0,7fcd58cfe066e299,mercantil,1,0,4.0,4.0,533.37,533.37,0.0,0.0,ATIVO,1,1.0,2.0,1.0,1.0,5,PRIMEIRA DIVIDA,2.14301
1,f1e9dde2a3053d51,imovel,4,0,79.0,79.0,7987.43,7987.43,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0,1,MEDIO INACESSIVEL,0.84104
2,5f225ff188d71bf6,imovel,1,0,12.0,12.0,454.28,454.28,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0,5,PRIMEIRA DIVIDA,2.14301
3,5f21929a71c17a08,mercantil,1,0,1.0,1.0,550.47,550.47,0.0,0.0,BAIXADO,1,0.0,0.0,1.0,1.0,5,PRIMEIRA DIVIDA,2.14301
4,196d80b1b82edbc0,mercantil,2,2,8.0,8.0,493.97,493.97,0.0,0.0,SUSPENSO,1,0.0,0.0,1.0,1.0,1,MEDIO INACESSIVEL,0.84104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494408,6dec5209fbc2f813,imovel,4,0,136.0,0.0,14185.80,0.00,0.0,1.0,ATIVO,1,2.0,3.0,0.0,0.0,1,MEDIO INACESSIVEL,0.84104
494409,6dec36900dea378c,mercantil,7,0,52.0,0.0,19190.68,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,0,CONTRIB PESSIMO,-0.79172
494410,6debc091e4bf4d84,mercantil,2,0,6.0,0.0,14930.45,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,1,MEDIO INACESSIVEL,0.84104
494411,6debbb4cc485f25f,mercantil,8,0,25.0,0.0,29849.66,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,0,CONTRIB PESSIMO,-0.79172


# Feature store do Contribuinte

In [99]:
df_feature_store_contribuinte = pessoas#.reset_index()
df_feature_store_contribuinte
# df_feature_store_contribuinte = df_feature_store_contribuinte[['id_pessoa', 'situacao', 'cpf_existe', 'edificacao', 'qtd_notas_2anos', 
#                                                                'situacao_ativa', 'status_situacao', 
#                                                                'deb_totais','deb_pagos', 'valor_tot', 'valor_pago', 
#                                                                'frequencia_da_pessoa', 'total_debitos_pessoa', 'debitos_pagos_pessoa', 'valor_total_pessoa', 'valor_pago_pessoa', 
#                                                                'historico_pagamento_em_qtd', 'historico_pagamento_em_valor', 
#                                                                'class_contribuinte', 'class_contribuinte_nome', 'class_contribuinte_peso']]

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,valor_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,situacao_ativa,status_situacao,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte,class_contribuinte_nome,class_contribuinte_peso
0,7fcd58cfe066e299,mercantil,1,0,4.0,4.0,533.37,533.37,0.0,0.0,ATIVO,1,1.0,2.0,1.0,1.0,5,PRIMEIRA DIVIDA,2.14301
1,f1e9dde2a3053d51,imovel,4,0,79.0,79.0,7987.43,7987.43,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0,1,MEDIO INACESSIVEL,0.84104
2,5f225ff188d71bf6,imovel,1,0,12.0,12.0,454.28,454.28,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0,5,PRIMEIRA DIVIDA,2.14301
3,5f21929a71c17a08,mercantil,1,0,1.0,1.0,550.47,550.47,0.0,0.0,BAIXADO,1,0.0,0.0,1.0,1.0,5,PRIMEIRA DIVIDA,2.14301
4,196d80b1b82edbc0,mercantil,2,2,8.0,8.0,493.97,493.97,0.0,0.0,SUSPENSO,1,0.0,0.0,1.0,1.0,1,MEDIO INACESSIVEL,0.84104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494408,6dec5209fbc2f813,imovel,4,0,136.0,0.0,14185.80,0.00,0.0,1.0,ATIVO,1,2.0,3.0,0.0,0.0,1,MEDIO INACESSIVEL,0.84104
494409,6dec36900dea378c,mercantil,7,0,52.0,0.0,19190.68,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,0,CONTRIB PESSIMO,-0.79172
494410,6debc091e4bf4d84,mercantil,2,0,6.0,0.0,14930.45,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,1,MEDIO INACESSIVEL,0.84104
494411,6debbb4cc485f25f,mercantil,8,0,25.0,0.0,29849.66,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,0,CONTRIB PESSIMO,-0.79172


In [100]:
df_feature_store_contribuinte.loc[df_feature_store_contribuinte['historico_pagamento_em_valor'] > 1, 'historico_pagamento_em_valor'] = 1
df_feature_store_contribuinte

,id_pessoa,tipo_divida,num_dist_cda,quantidade_reparcelamento,deb_totais,deb_pagos,valor_tot,valor_pago,qtd_notas_2anos,edificacao,situacao,cpf_cnpj_existe,situacao_ativa,status_situacao,historico_pagamento_em_qtd,historico_pagamento_em_valor,class_contribuinte,class_contribuinte_nome,class_contribuinte_peso
0,7fcd58cfe066e299,mercantil,1,0,4.0,4.0,533.37,533.37,0.0,0.0,ATIVO,1,1.0,2.0,1.0,1.0,5,PRIMEIRA DIVIDA,2.14301
1,f1e9dde2a3053d51,imovel,4,0,79.0,79.0,7987.43,7987.43,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0,1,MEDIO INACESSIVEL,0.84104
2,5f225ff188d71bf6,imovel,1,0,12.0,12.0,454.28,454.28,0.0,1.0,ATIVO,1,2.0,3.0,1.0,1.0,5,PRIMEIRA DIVIDA,2.14301
3,5f21929a71c17a08,mercantil,1,0,1.0,1.0,550.47,550.47,0.0,0.0,BAIXADO,1,0.0,0.0,1.0,1.0,5,PRIMEIRA DIVIDA,2.14301
4,196d80b1b82edbc0,mercantil,2,2,8.0,8.0,493.97,493.97,0.0,0.0,SUSPENSO,1,0.0,0.0,1.0,1.0,1,MEDIO INACESSIVEL,0.84104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494408,6dec5209fbc2f813,imovel,4,0,136.0,0.0,14185.80,0.00,0.0,1.0,ATIVO,1,2.0,3.0,0.0,0.0,1,MEDIO INACESSIVEL,0.84104
494409,6dec36900dea378c,mercantil,7,0,52.0,0.0,19190.68,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,0,CONTRIB PESSIMO,-0.79172
494410,6debc091e4bf4d84,mercantil,2,0,6.0,0.0,14930.45,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,1,MEDIO INACESSIVEL,0.84104
494411,6debbb4cc485f25f,mercantil,8,0,25.0,0.0,29849.66,0.00,0.0,0.0,INAPTO,1,0.0,0.0,0.0,0.0,0,CONTRIB PESSIMO,-0.79172


In [101]:
df_feature_store_contribuinte.groupby('class_contribuinte_nome')['id_pessoa'].nunique().to_frame().reset_index()

,class_contribuinte_nome,id_pessoa
0,CONTRIB EXCELENTE,33381
1,CONTRIB NEGOCIADOR,4787
2,CONTRIB PESSIMO,106010
3,MEDIO INACESSIVEL,165172
4,PRIMEIRA DIVIDA,100601
5,RUIM ACESSIVEL,68854


# Salvar no S3

In [108]:
print("Inicia a conexão com S3 para inscrição dos dados")
# Cria conexão ao s3 e preenche a tabela com os dados
s3_resource = boto3.resource(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id=os.getenv("AWS_ACESS_KEY"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACESS_KEY")
    )

NOME_ARQ_SALVAR_S3 = 'feature_store_contribuinte_prime.csv'

up_s3_files(dataframe=df_feature_store_contribuinte, 
            bucket_name=os.getenv("S3_BUCKET_NAME"), 
            folder_name=os.getenv("S3_FOLDER_NAME"), 
            file_name= NOME_ARQ_SALVAR_S3)

print("Dados upados no s3")
print("Processo finalizado")

Inicia a conexão com S3 para inscrição dos dados
Dados upados no s3
Processo finalizado
